In [136]:
import pandas as pd
import numpy as np
import os

import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, dash_table
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import math


In [137]:
# Calculating the distance between two points on Earth, using the Haversine formula
"""def haversine_km(lat1, lon1, lat2, lon2):
    # Earth radius in kilometers
    R = 6371.0  
    lat1_rad = math.radians(lat1)
    lat2_rad = math.radians(lat2)
    delta_lat = math.radians(lat2 - lat1)
    delta_lon = math.radians(lon2 - lon1)
    # Calculating differences
    a = math.sin(delta_lat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2)**2
    return 2 * R * math.asin(math.sqrt(a))

# Streets names
locations = {
    "Saaritie":        (62.136788, 25.762473),
    "Tuulimyllyntie":  (62.221789, 25.695931),
    "Tähtiniementie":  (62.011127, 25.552755),
    "Kaakkovuorentie": (62.294362, 25.800196),
    "Kotaniementie":   (62.265705, 25.909542),}

# distance matrix
location_names = list(locations.keys())
distance_matrix = pd.DataFrame(index=location_names, columns=location_names)

for loc1 in location_names:
    for loc2 in location_names:
        if loc1 == loc2:
            distance_matrix.loc[loc1, loc2] = 0.0
        else:
            lat1, lon1 = locations[loc1]
            lat2, lon2 = locations[loc2]
            distance_km = haversine_km(lat1, lon1, lat2, lon2)
            distance_matrix.loc[loc1, loc2] = round(distance_km, 1)

print("Distance Matrix (km):")
print(distance_matrix)
"""

'def haversine_km(lat1, lon1, lat2, lon2):\n    # Earth radius in kilometers\n    R = 6371.0  \n    lat1_rad = math.radians(lat1)\n    lat2_rad = math.radians(lat2)\n    delta_lat = math.radians(lat2 - lat1)\n    delta_lon = math.radians(lon2 - lon1)\n    # Calculating differences\n    a = math.sin(delta_lat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon/2)**2\n    return 2 * R * math.asin(math.sqrt(a))\n\n# Streets names\nlocations = {\n    "Saaritie":        (62.136788, 25.762473),\n    "Tuulimyllyntie":  (62.221789, 25.695931),\n    "Tähtiniementie":  (62.011127, 25.552755),\n    "Kaakkovuorentie": (62.294362, 25.800196),\n    "Kotaniementie":   (62.265705, 25.909542),}\n\n# distance matrix\nlocation_names = list(locations.keys())\ndistance_matrix = pd.DataFrame(index=location_names, columns=location_names)\n\nfor loc1 in location_names:\n    for loc2 in location_names:\n        if loc1 == loc2:\n            distance_matrix.loc[loc1, loc2] = 0.0\n        else:\

<h2><center>Kaakkovuorentie<h2>

In [138]:
base = r"D:\Fiveth\Project_semester_automn_2025\Marjetas_Data\Marjetas_Data\JKL WS100\Data\Kaakkovuorentie"
files = os.listdir(base)
files

['Kaakkovuorentie_202404-202406.csv',
 'Kaakkovuorentie_202407-202412.csv',
 'Kaakkovuorentie_202501-202506.csv',
 'Kaakkovuorentie_202507-202509.csv']

In [139]:
file_path = [os.path.join(base, f) for f in os.listdir(base) if f.endswith('.csv')]
Kaakkovuorentie = pd.concat((pd.read_csv(f, sep=';') for f in file_path))
Kaakkovuorentie["Timestamp"] = pd.to_datetime(Kaakkovuorentie["Timestamp"],format="%Y-%m-%d %H:%M:%S")
print("NaT after parse:", Kaakkovuorentie["Timestamp"].isna().sum())
# Sort the ds by timestamp
Kaakkovuorentie = Kaakkovuorentie.sort_values(by="Timestamp").reset_index(drop=True)

NaT after parse: 0


In [140]:
Kaakkovuorentie.head()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2024-04-02 08:15:00,0.0,0.0,36.78,0.0,0
1,2024-04-02 08:25:02,0.0,0.0,36.78,0.0,0
2,2024-04-02 08:35:00,0.0,0.0,36.78,0.0,0
3,2024-04-02 08:45:02,0.0,0.0,36.78,0.0,0
4,2024-04-02 08:55:03,0.0,0.0,36.78,0.0,0


In [141]:
Kaakkovuorentie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101772 entries, 0 to 101771
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Timestamp                      101772 non-null  datetime64[ns]
 1   precipitationIntensity_mm_h    101772 non-null  float64       
 2   precipitationIntensity_mm_min  101772 non-null  float64       
 3   precipitationQuantityAbs_mm    101772 non-null  float64       
 4   precipitationQuantityDiff_mm   101772 non-null  float64       
 5   precipitationType              101772 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 4.7 MB


In [142]:
Kaakkovuorentie['Timestamp'] = Kaakkovuorentie['Timestamp'].dt.round("min")
Kaakkovuorentie.shape

(101772, 6)

In [143]:
Kaakkovuorentie.head()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2024-04-02 08:15:00,0.0,0.0,36.78,0.0,0
1,2024-04-02 08:25:00,0.0,0.0,36.78,0.0,0
2,2024-04-02 08:35:00,0.0,0.0,36.78,0.0,0
3,2024-04-02 08:45:00,0.0,0.0,36.78,0.0,0
4,2024-04-02 08:55:00,0.0,0.0,36.78,0.0,0


In [144]:
Kaakkovuorentie.describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,101772,101772.000000,101772.000000,101772.000000,101772.000000,101772.000000
mean,2025-01-04 18:43:53.602169600,0.076001,0.001264,106.562061,0.008142,10.046290
min,2024-04-02 08:15:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2024-08-28 09:25:00,0.000000,0.000000,54.750000,0.000000,0.000000
50%,2025-01-08 10:39:00,0.000000,0.000000,78.620000,0.000000,0.000000
75%,2025-05-15 11:05:30,0.000000,0.000000,133.740000,0.000000,0.000000
max,2025-09-18 09:49:00,111.300000,1.855000,377.630000,9.850000,70.000000
std,NaN,0.807998,0.013466,79.367265,0.105612,23.927157


In [145]:
# Let's find out start and end time of the data and the total days 
start_time = Kaakkovuorentie["Timestamp"].min()
end_time = Kaakkovuorentie["Timestamp"].max()
total_days = (end_time - start_time).days
print(f"Data starts from: {start_time}")
print(f"Data ends at: {end_time}")
print(f"Total days of data: {total_days} days")

Data starts from: 2024-04-02 08:15:00
Data ends at: 2025-09-18 09:49:00
Total days of data: 534 days


In [146]:
# Check for duplicates
Kaakkovuorentie.duplicated().sum()


np.int64(28270)

In [147]:
# printing the duplicated rows
duplicates = Kaakkovuorentie[Kaakkovuorentie.duplicated()]
duplicates

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
2997,2024-04-23 05:40:00,0.0,0.0,64.28,0.0,0
2998,2024-04-23 05:40:00,0.0,0.0,64.28,0.0,0
2999,2024-04-23 05:40:00,0.0,0.0,64.28,0.0,0
3000,2024-04-23 05:40:00,0.0,0.0,64.28,0.0,0
3001,2024-04-23 05:40:00,0.0,0.0,64.28,0.0,0
...,...,...,...,...,...,...
95008,2025-08-02 10:58:00,0.0,0.0,250.47,0.0,0
95009,2025-08-02 10:58:00,0.0,0.0,250.47,0.0,0
95010,2025-08-02 10:58:00,0.0,0.0,250.47,0.0,0
95011,2025-08-02 10:58:00,0.0,0.0,250.47,0.0,0


In [148]:
# Remove duplicates
Kaakkovuorentie = Kaakkovuorentie.drop_duplicates()
Kaakkovuorentie.shape


(73502, 6)

In [149]:
def check_timestamp_issues(df, Kaakkovuorentie, expected_minutes=10):
    print(f"Checking: {Kaakkovuorentie}")
    

    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')
    
    # Calculate time differences
    time_diffs = df['Timestamp'].diff()
    expected_gap = pd.Timedelta(minutes=expected_minutes)
    
    # Check for issues in Timestamps
    large_gaps = time_diffs[time_diffs > expected_gap * 1.1]
    duplicates = df['Timestamp'].duplicated().sum()
    time_travel = (time_diffs < pd.Timedelta(0)).sum()
    
    df['Date'] = df['Timestamp'].dt.date
    daily_counts = df['Date'].value_counts().sort_index()
   
    # Print results
    print(f"Number of gaps larger than {expected_minutes} minutes: {len(large_gaps)}")
    print(f"Large gaps: {large_gaps.max()}")
    print(f"Duplicates: {duplicates}")
    print(f"Time travel: {time_travel}")
    print(f"Daily counts: {daily_counts}")
   
Kaakkovuorentie_info = check_timestamp_issues(Kaakkovuorentie, "Kaakkovuorentie WS100")

Checking: Kaakkovuorentie WS100
Number of gaps larger than 10 minutes: 138
Large gaps: 20 days 03:47:00
Duplicates: 16
Time travel: 0
Daily counts: Date
2024-04-02     95
2024-04-03    143
2024-04-04    142
2024-04-05    143
2024-04-06    144
             ... 
2025-09-14    144
2025-09-15    144
2025-09-16    144
2025-09-17    144
2025-09-18     59
Name: count, Length: 515, dtype: int64


In [150]:
Kaakkovuorentie['precipitationType'].unique()

array([ 0, 67, 70, 69, 60])

**Dry**

In [151]:
Kaakkovuorentie[Kaakkovuorentie['precipitationType'] == 0].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,65189,65189.000000,65189.000000,65189.000000,65189.000000,65189.0
mean,2024-12-29 08:06:28.592247296,0.000104,0.000002,105.738532,0.000326,0.0
min,2024-04-02 08:15:00,0.000000,0.000000,0.000000,0.000000,0.0
25%,2024-08-08 17:19:00,0.000000,0.000000,52.960000,0.000000,0.0
50%,2025-01-06 19:43:00,0.000000,0.000000,78.620000,0.000000,0.0
75%,2025-05-14 04:43:00,0.000000,0.000000,133.730000,0.000000,0.0
max,2025-09-18 09:49:00,0.100000,0.002000,377.630000,1.150000,0.0
std,NaN,0.003228,0.000065,84.983797,0.009467,0.0


**Snowy**

In [152]:
Kaakkovuorentie[Kaakkovuorentie['precipitationType'] == 70].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,1437,1437.000000,1437.000000,1437.000000,1437.000000,1437.0
mean,2024-12-13 01:22:25.762004224,0.265484,0.004462,62.363333,0.042408,70.0
min,2024-04-07 08:46:00,0.000000,0.000000,0.000000,0.000000,70.0
25%,2024-11-25 03:00:00,0.000000,0.000000,30.960000,0.010000,70.0
50%,2025-01-13 19:35:00,0.200000,0.003000,63.360000,0.020000,70.0
75%,2025-02-26 02:29:00,0.400000,0.007000,88.710000,0.070000,70.0
max,2025-05-22 06:05:00,1.500000,0.025000,134.220000,0.250000,70.0
std,NaN,0.291605,0.004834,35.769337,0.046354,0.0


**Rain**

In [153]:
Kaakkovuorentie[Kaakkovuorentie['precipitationType'] == 60].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,4237,4237.000000,4237.000000,4237.000000,4237.000000,4237.0
mean,2025-01-30 18:32:41.958933248,0.899103,0.015034,141.388518,0.148225,60.0
min,2024-04-08 15:26:00,0.000000,0.000000,0.000000,0.000000,60.0
25%,2024-08-03 21:38:00,0.000000,0.000000,64.500000,0.000000,60.0
50%,2025-04-28 23:30:00,0.100000,0.002000,121.650000,0.020000,60.0
75%,2025-06-30 06:27:00,0.400000,0.007000,210.840000,0.090000,60.0
max,2025-09-17 21:59:00,111.300000,1.855000,377.610000,9.850000,60.0
std,NaN,3.806719,0.063434,98.022941,0.491737,0.0


**Mixe(Snow and rain)**

In [154]:
Kaakkovuorentie[(Kaakkovuorentie['precipitationType'] >= 60) & (Kaakkovuorentie['precipitationType'] <= 69)].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,6876,6876.000000,6876.000000,6876.000000,6876.000000,6876.000000
mean,2025-01-03 22:22:02.757417472,0.662856,0.011096,112.962629,0.108562,62.846859
min,2024-04-05 17:06:00,0.000000,0.000000,0.000000,0.000000,60.000000
25%,2024-08-16 18:48:30,0.000000,0.000000,53.067500,0.010000,60.000000
50%,2024-12-29 05:53:00,0.100000,0.002000,85.810000,0.020000,60.000000
75%,2025-06-02 13:20:30,0.400000,0.007000,156.152500,0.070000,67.000000
max,2025-09-17 21:59:00,111.300000,1.855000,377.610000,9.850000,69.000000
std,NaN,3.014793,0.050237,87.786205,0.390678,3.642491


In [135]:
# Save the cleaned data
Kaakkovuorentie.to_csv(r"D:\Fiveth\Marjetas_project\cleaned_datasets\wes100\df_Kaakkovuorentie.csv", index=False)

<h3><center>Exploring Kaakkovuorentie<h3>

In [22]:
# Mapping precipitationType codes to labels
def map_pt(x):
    if x == 0:
        return 'dry'
    if x == 70:
        return 'snow'
    if x == 60:
        return 'rain'
    if 61 <= x <= 69:
        return 'mixed'  

In [23]:
df_Kaakkovuorentie = Kaakkovuorentie.copy() 

df_Kaakkovuorentie['ptype'] = df_Kaakkovuorentie['precipitationType'].apply(map_pt)
df_Kaakkovuorentie['is_wet'] = df_Kaakkovuorentie['ptype'].ne('dry')

In [24]:
df_Kaakkovuorentie.head()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType,ptype,is_wet
0,2024-04-02 08:15:00,0.0,0.0,36.78,0.0,0,dry,False
1,2024-04-02 08:25:00,0.0,0.0,36.78,0.0,0,dry,False
2,2024-04-02 08:35:00,0.0,0.0,36.78,0.0,0,dry,False
3,2024-04-02 08:45:00,0.0,0.0,36.78,0.0,0,dry,False
4,2024-04-02 08:55:00,0.0,0.0,36.78,0.0,0,dry,False


**Grouping and aggregation:**

In [25]:
summary = (df_Kaakkovuorentie.groupby('ptype').agg(
          rows=('ptype', 'size'),
          # Converting records to hours, each record is 10 minutes 
          hours=('ptype', lambda s: s.size/6.0),
          # Total precipitation in mm for each type
          total_mm=('precipitationQuantityDiff_mm', 'sum'),
          # Average precipitation per record
          mean_mm_per_record=('precipitationQuantityDiff_mm', 'mean'),
          # Intensity distribution
          p25_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 25)),
          p50_intensity=('precipitationIntensity_mm_h', 'median'),
          p75_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 75)),
          p99_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 99)),).sort_values('total_mm', ascending=False).round(1))
summary

,rows,hours,total_mm,mean_mm_per_record,p25_intensity,p50_intensity,p75_intensity,p99_intensity
ptype,,,,,,,,
rain,4237,706.2,628.0,0.1,0.0,0.1,0.4,12.6
mixed,2639,439.8,118.4,0.0,0.1,0.2,0.4,1.9
snow,1437,239.5,60.9,0.0,0.0,0.2,0.4,1.1
dry,65189,10864.8,21.2,0.0,0.0,0.0,0.0,0.0


**Finding Precipitation events**

In [26]:
grouping = (df_Kaakkovuorentie['is_wet'] != df_Kaakkovuorentie['is_wet'].shift()).cumsum()

# Event aggregation:
events = (df_Kaakkovuorentie[df_Kaakkovuorentie['is_wet']]
    .assign(event_id=grouping[df_Kaakkovuorentie['is_wet']])
    #Grouping  by each event
    .groupby('event_id').agg(
          # Event start time
        start=('Timestamp','min'),
        # Event end time
        end=('Timestamp','max'),
        duration_h=('Timestamp', lambda x: (x.max()-x.min()).total_seconds()/3600.0),
        total_mm=('precipitationQuantityDiff_mm','sum'),
        peak_intensity_mm_h=('precipitationIntensity_mm_h','max'),
        dominant_ptype=('ptype', lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0]),
        n=('ptype','size'))
    .sort_values(['total_mm','duration_h'], ascending=[False, False])
    .reset_index(drop=True))
event_stats = (events.groupby('dominant_ptype').agg(
              events=('dominant_ptype','size'),
              median_duration_h=('duration_h','median'),
              median_total_mm=('total_mm','median'),
              top_event_total_mm=('total_mm', lambda x: x.nlargest(10).sum()),
              total_mm=('total_mm','sum')).sort_values('total_mm', ascending=False).round(1))
events.head(25)
# Kaakkovuorentie 

,start,end,duration_h,total_mm,peak_intensity_mm_h,dominant_ptype,n
0,2024-08-30 20:37:00,2024-08-30 23:27:00,2.833333,40.44,81.4,rain,18
1,2025-09-09 04:57:00,2025-09-09 08:17:00,3.333333,35.91,45.0,rain,21
2,2024-06-09 13:21:00,2024-06-09 17:01:00,3.666667,16.13,18.1,rain,23
3,2025-06-24 16:26:00,2025-06-25 07:26:00,15.000000,15.19,9.3,rain,91
4,2024-09-13 09:40:00,2024-09-13 14:20:00,4.666667,15.04,11.1,rain,28
5,2025-05-29 19:07:00,2025-05-29 22:47:00,3.666667,14.55,29.4,rain,23
6,2024-09-15 00:51:00,2024-09-15 07:01:00,6.166667,13.02,12.8,rain,38
7,2025-06-28 23:27:00,2025-06-29 04:17:00,4.833333,12.19,8.8,rain,30
8,2025-09-17 09:59:00,2025-09-17 12:59:00,3.000000,12.16,17.5,rain,19
9,2024-08-12 03:50:00,2024-08-12 05:50:00,2.000000,11.78,30.3,rain,13


In [27]:
event_stats

,events,median_duration_h,median_total_mm,top_event_total_mm,total_mm
dominant_ptype,,,,,
rain,990,0.2,0.0,186.4,626.5
mixed,423,0.2,0.0,49.9,130.9
snow,249,0.2,0.0,31.6,50.0


In [28]:
# Monthly aggregation
df_Kaakkovuorentie['month'] = df_Kaakkovuorentie['Timestamp'].dt.to_period('M')
monthly = (
    df_Kaakkovuorentie.groupby(['month','ptype']).agg(total_mm=('precipitationQuantityDiff_mm','sum'), hours=('ptype', 'size')).reset_index())

# Hour of day 
df_Kaakkovuorentie['hour'] = df_Kaakkovuorentie['Timestamp'].dt.hour
hour_day = (df_Kaakkovuorentie[df_Kaakkovuorentie['is_wet']].groupby(['hour','ptype']).agg(mm_per_hour=('precipitationQuantityDiff_mm','sum'),
        records=('ptype','size')).reset_index())

In [29]:
monthly.head(50)

,month,ptype,total_mm,hours
0,2024-04,dry,0.66,3328
1,2024-04,mixed,27.63,483
2,2024-04,rain,6.54,91
3,2024-04,snow,6.93,213
4,2024-05,dry,1.33,4321
5,2024-05,mixed,0.63,29
6,2024-05,rain,20.51,91
7,2024-05,snow,0.12,6
8,2024-06,dry,0.78,4017
9,2024-06,rain,41.80,280


In [30]:
hour_day.head(50)

,hour,ptype,mm_per_hour,records
0,0,mixed,5.00,103
1,0,rain,24.13,176
2,0,snow,2.08,58
3,1,mixed,4.61,86
4,1,rain,21.86,143
5,1,snow,3.21,67
6,2,mixed,4.44,96
7,2,rain,32.45,160
8,2,snow,3.67,65
9,3,mixed,4.16,103


**precipitation transition analysis**

In [31]:
df_Kaakkovuorentie['ptype_next'] = df_Kaakkovuorentie['ptype'].shift(-1).fillna('dry')
transitions = (df_Kaakkovuorentie.groupby(['ptype','ptype_next']).size().unstack(fill_value=0))

transitions_prob = transitions.div(transitions.sum(axis=1), axis=0).round(3)
transitions_prob


ptype_next,dry,mixed,rain,snow
ptype,,,,
dry,0.975,0.006,0.015,0.004
mixed,0.145,0.667,0.003,0.185
rain,0.233,0.003,0.755,0.009
snow,0.204,0.338,0.024,0.435


In [32]:
# Extremes Outliers per type
exterm = (df_Kaakkovuorentie[df_Kaakkovuorentie['is_wet']]
       .groupby('ptype')['precipitationIntensity_mm_h']
       .quantile([0.95,0.99,0.999]).unstack().round(1))


cut = df_Kaakkovuorentie['precipitationIntensity_mm_h'].quantile(0.999)
spikes = df_Kaakkovuorentie[df_Kaakkovuorentie['precipitationIntensity_mm_h'] > cut]

# Optional: cap intensities to 99.9% to stabilize stats
df_Kaakkovuorentie['intensity_capped'] = np.minimum(df_Kaakkovuorentie['precipitationIntensity_mm_h'], cut)
df_Kaakkovuorentie

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType,ptype,is_wet,month,hour,ptype_next,intensity_capped
0,2024-04-02 08:15:00,0.0,0.0,36.78,0.0,0,dry,False,2024-04,8,dry,0.0
1,2024-04-02 08:25:00,0.0,0.0,36.78,0.0,0,dry,False,2024-04,8,dry,0.0
2,2024-04-02 08:35:00,0.0,0.0,36.78,0.0,0,dry,False,2024-04,8,dry,0.0
3,2024-04-02 08:45:00,0.0,0.0,36.78,0.0,0,dry,False,2024-04,8,dry,0.0
4,2024-04-02 08:55:00,0.0,0.0,36.78,0.0,0,dry,False,2024-04,8,dry,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
101767,2025-09-18 09:09:00,0.0,0.0,377.63,0.0,0,dry,False,2025-09,9,dry,0.0
101768,2025-09-18 09:19:00,0.0,0.0,377.63,0.0,0,dry,False,2025-09,9,dry,0.0
101769,2025-09-18 09:29:00,0.0,0.0,377.63,0.0,0,dry,False,2025-09,9,dry,0.0
101770,2025-09-18 09:39:00,0.0,0.0,377.63,0.0,0,dry,False,2025-09,9,dry,0.0


In [33]:
report = (
    summary[['rows','hours','total_mm','mean_mm_per_record','p50_intensity','p99_intensity']]
    .rename(columns={
        'rows':'records',
        'hours':'hours_logged',
        'total_mm':'total_precip_mm',
        'mean_mm_per_record':'mean_mm_per_10min',
        'p50_intensity':'median_intensity_mm_h',
        'p99_intensity':'p99_intensity_mm_h'}).round(1))
report


,records,hours_logged,total_precip_mm,mean_mm_per_10min,median_intensity_mm_h,p99_intensity_mm_h
ptype,,,,,,
rain,4237,706.2,628.0,0.1,0.1,12.6
mixed,2639,439.8,118.4,0.0,0.2,1.9
snow,1437,239.5,60.9,0.0,0.2,1.1
dry,65189,10864.8,21.2,0.0,0.0,0.0


In [34]:
# 11.5% of the whole record 1439.2 h wet out of 12466

In [35]:
Checking_data_Kaakkovuorentie = Kaakkovuorentie.copy()
Checking_data_Kaakkovuorentie['Timestamp'] = pd.to_datetime(Checking_data_Kaakkovuorentie['Timestamp'], errors='coerce')
Checking_data_Kaakkovuorentie = Checking_data_Kaakkovuorentie.sort_values('Timestamp').set_index('Timestamp')

# negative rain increments usually should not  happen
bad_neg = Checking_data_Kaakkovuorentie['precipitationQuantityDiff_mm'] < -0.01

# impossible crazy bursts
bad_spike = Checking_data_Kaakkovuorentie['precipitationIntensity_mm_h'] > 100

# check gaps in data
dt = Checking_data_Kaakkovuorentie.index.to_series().diff().dt.total_seconds()/60.0
long_gaps = dt[dt > 60]

print("Number of negative rain increments:", bad_neg.sum())
print("Number of spikes in precipitation intensity:", bad_spike.sum())
print("Number of long gaps (>1h):", long_gaps.shape[0])

print("\nActual negative precipitation values:")
# Display the actual rows with negative precipitation
# Show actual problematic values
if bad_neg.any():
    print("Negative precipitation:")
    print(Checking_data_Kaakkovuorentie[bad_neg]['precipitationQuantityDiff_mm'].to_string())

if bad_spike.any():
    print("\nRain spikes:")
    print(Checking_data_Kaakkovuorentie[bad_spike]['precipitationIntensity_mm_h'].to_string())

if long_gaps.any():
    print("\nGap durations (minutes):")
    gaps = Checking_data_Kaakkovuorentie.index.to_series().diff().dt.total_seconds()/60
    print(gaps[gaps > 60].to_string())

Number of negative rain increments: 0
Number of spikes in precipitation intensity: 1
Number of long gaps (>1h): 8

Actual negative precipitation values:

Rain spikes:
Timestamp
2024-05-30 15:48:00    111.3

Gap durations (minutes):
Timestamp
2024-07-31 13:57:00     2950.0
2024-08-04 04:00:00      152.0
2024-09-03 00:18:00       70.0
2024-09-09 07:09:00      370.0
2024-09-14 15:21:00      160.0
2024-09-24 01:50:00      120.0
2024-10-15 14:28:00    29027.0
2025-03-30 04:10:00       70.0


In [159]:
data_Kaakkovuorentie = Kaakkovuorentie.copy()

def map_event(code: int):
    # rules
    if code == 0:
        return "None"
    if code == 60:
        return "Rain"
    if 61 <= code <= 69:
        return "Mix"
    if code == 70:
        return "Snow"
    return "Other"

def prepare(df_raw: pd.DataFrame) -> pd.DataFrame:
    data_Kaakkovuorentie = df_raw.copy()
    data_Kaakkovuorentie["precipitationQuantityDiff_mm"] = data_Kaakkovuorentie["precipitationQuantityDiff_mm"].clip(lower=0)
    data_Kaakkovuorentie["event"] = data_Kaakkovuorentie["precipitationType"].fillna(0).astype(int).map(map_event)

    # duration
    dt = data_Kaakkovuorentie["Timestamp"].shift(-1) - data_Kaakkovuorentie["Timestamp"]
    dt_sec = dt.dt.total_seconds()

    # Fill last/invalid intervals with median positive step
    median_step = np.nanmedian(dt_sec[dt_sec > 0]) if np.any(dt_sec > 0) else 60.0
    dt_sec = np.where((dt_sec <= 0) | np.isnan(dt_sec), median_step, dt_sec)

    data_Kaakkovuorentie["duration_h"] = dt_sec / 3600.0
    data_Kaakkovuorentie.set_index("Timestamp", inplace=True)
    return data_Kaakkovuorentie

data_Kaakkovuorentie = prepare(data_Kaakkovuorentie)

# Helper: aggregate by freq within range
FREQS = {"D": "Daily", "W-MON": "Weekly (Mon start)", "MS": "Monthly (Start)"}

def aggregate(data_: pd.DataFrame, start: str, end: str, freq: str):
    sub = data_Kaakkovuorentie.loc[start:end]
    g = pd.Grouper(freq=freq)

    # Duration by event
    dur_event = (
        sub.groupby([g, "event"])["duration_h"]
           .sum().reset_index()
    )

    # Precip by event
    mm_event = (
        sub.groupby([g, "event"])["precipitationQuantityDiff_mm"]
           .sum().reset_index()
    )

    # Total precip (for trend)
    mm_total = (
        sub["precipitationQuantityDiff_mm"]
           .resample(freq).sum().reset_index()
           .rename(columns={"precipitationQuantityDiff_mm":"total_mm", "Timestamp":"period"})
    )

    # Share % by event (time share)
    dur_total = dur_event.groupby("Timestamp")["duration_h"].sum().rename("dur_total")
    dur_event = dur_event.merge(dur_total, on="Timestamp", how="left")
    dur_event["share_%"] = np.where(dur_event["dur_total"] > 0,
                                    100 * dur_event["duration_h"] / dur_event["dur_total"], 0)

    # Round for display
    for col in ["duration_h", "share_%"]:
        dur_event[col] = dur_event[col].round(1)
    mm_event["precip_mm"] = mm_event["precipitationQuantityDiff_mm"].round(1)
    mm_event.drop(columns=["precipitationQuantityDiff_mm"], inplace=True)
    mm_total["total_mm"] = mm_total["total_mm"].round(1)

    return dur_event, mm_event, mm_total

# Dash app
app = Dash(__name__)
min_date = data_Kaakkovuorentie.index.min().date()
max_date = data_Kaakkovuorentie.index.max().date()

app.layout = html.Div(
    style={"fontFamily":"system-ui","padding":"16px","maxWidth":"1200px","margin":"0 auto"},
    children=[
        html.H2("WS100 – Dynamic Precipitation Dashboard"),
        html.Div([
            html.Div([
                html.Label("Date range"),
                dcc.DatePickerRange(
                    id="date-range", start_date=min_date, end_date=max_date,
                    min_date_allowed=min_date, max_date_allowed=max_date
                ),
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Aggregation"),
                dcc.Dropdown(
                    id="freq", value="MS",
                    options=[{"label":v, "value":k} for k,v in FREQS.items()],
                    clearable=False, style={"width":"220px"}
                )
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Metric"),
                dcc.RadioItems(
                    id="metric", value="duration",
                    options=[
                        {"label":"Duration (hours)", "value":"duration"},
                        {"label":"Precipitation (mm)", "value":"precip"},
                        {"label":"Time share (%)", "value":"share"},
                    ],
                    inline=True
                )
            ], style={"display":"inline-block"}),
        ], style={"marginBottom":"12px"}),

        dcc.Graph(id="stacked-graph", style={"height":"480px"}),
        dcc.Graph(id="total-graph", style={"height":"360px"}),

        html.H3("Summary (by period × event)"),
        dash_table.DataTable(
            id="summary-table",
            page_size=12,
            style_table={"overflowX":"auto"},
            style_cell={"padding":"6px", "fontSize":"14px"},
            style_header={"fontWeight":"600"}
        ),
        html.Div("Notes: Duration is computed from the time difference to the next timestamp; "
                 "‘Share’ is duration share per period. Precip negatives are clipped to 0.",
                 style={"marginTop":"8px","color":"#555"})
    ]
)

# ------------------------------------------------------------
# Callbacks
@app.callback(
    [Output("stacked-graph","figure"),
     Output("total-graph","figure"),
     Output("summary-table","data"),
     Output("summary-table","columns")],
    [Input("date-range","start_date"),
     Input("date-range","end_date"),
     Input("freq","value"),
     Input("metric","value")]
)
def update(start_date, end_date, freq, metric):
    dur_event, mm_event, mm_total = aggregate(data_Kaakkovuorentie, start_date, end_date, freq)

    # Build stacked bar data & labels
    if metric == "duration":
        ycol, ytitle = "duration_h", "Duration (hours)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})
    elif metric == "precip":
        ycol, ytitle = "precip_mm", "Precipitation (mm)"
        df_plot = mm_event.rename(columns={"Timestamp":"period"})
    else:  # share
        ycol, ytitle = "share_%", "Time share (%)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})

    fig_stack = px.bar(
        df_plot, x="period", y=ycol, color="event",
        barmode="stack", text_auto=".1f",
        labels={"period":"Period", ycol:ytitle, "event":"Event"},
        title=f"{FREQS.get(freq, freq)} — {ytitle}"
    )
    if metric == "share":
        fig_stack.update_yaxes(range=[0, 100])

    # Total precip trend
    fig_total = px.line(
        mm_total, x="period", y="total_mm",
        markers=True, labels={"period":"Period","total_mm":"Total Precip (mm)"},
        title=f"{FREQS.get(freq, freq)} — Total Precipitation"
    )

    # Summary table: pivot by period × event with both duration and precip
    # Merge duration + precip into one wide table
    pivot_dur = dur_event.pivot_table(index="Timestamp", columns="event", values="duration_h", fill_value=0)
    pivot_mm  = mm_event.pivot_table(index="Timestamp", columns="event", values="precip_mm",  fill_value=0)
    out = pd.concat(
        {"Dur(h)": pivot_dur, "MM": pivot_mm},
        axis=1
    ).reset_index().rename(columns={"Timestamp":"period"})

    # Round nicely and format period
    out = out.sort_values("period")
    out["period"] = out["period"].dt.strftime("%Y-%m-%d")

    columns = [{"name": col if isinstance(col, str) else " / ".join(col), "id": col if isinstance(col, str) else " / ".join(col)}
               for col in out.columns]
    out.columns = [c["id"] for c in columns]
    data = out.round(1).to_dict("records")

    return fig_stack, fig_total, data, columns


# ------------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True)


In [37]:
# Find the max value in the precipitationQuantityDiff_mm column
max_precipitation = Kaakkovuorentie["precipitationQuantityDiff_mm"].max()
max_precipitation

np.float64(9.85)

<h2><center>Kotaniementie<h3>

In [38]:
base = r"D:\Fiveth\Project_semester_automn_2025\Marjetas_Data\Marjetas_Data\JKL WS100\Data\Kotaniementie"
files = os.listdir(base)
files

['Kotaniementie_202101-202106.csv',
 'Kotaniementie_202107-202112.csv',
 'Kotaniementie_202201-202206.csv',
 'Kotaniementie_202207-202212.csv',
 'Kotaniementie_202301-202306.csv',
 'Kotaniementie_202306-202312.csv',
 'Kotaniementie_202401-202406.csv',
 'Kotaniementie_202406-202412.csv',
 'Kotaniementie_202501-202509.csv']

In [39]:
file_path = [os.path.join(base, f) for f in os.listdir(base) if f.endswith('.csv')]
Kotaniementie = pd.concat((pd.read_csv(f, sep=';') for f in file_path), ignore_index=True)


Kotaniementie["Timestamp"] = pd.to_datetime(Kotaniementie["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", Kotaniementie["Timestamp"].isna().sum())
# Sort the ds by timestamp
Kotaniementie = Kotaniementie.sort_values(by="Timestamp").reset_index(drop=True)

NaT after parse: 0


In [40]:
Kotaniementie.head()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2021-01-01 00:03:44,0.0,0.000,250.46,0.00,0
1,2021-01-01 00:13:44,0.1,0.002,250.46,0.00,70
2,2021-01-01 00:23:44,0.1,0.002,250.48,0.02,70
3,2021-01-01 00:33:45,0.1,0.002,250.49,0.01,70
4,2021-01-01 00:43:44,0.1,0.002,250.51,0.02,70


In [41]:
Kotaniementie.tail()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
330114,2025-09-18 09:26:26,0.0,0.0,208.16,0.0,0
330115,2025-09-18 09:36:26,0.0,0.0,208.16,0.0,0
330116,2025-09-18 09:46:27,0.0,0.0,208.16,0.0,0
330117,2025-09-18 09:56:27,0.0,0.0,208.16,0.0,0
330118,2025-09-18 10:06:27,0.0,0.0,208.16,0.0,0


In [42]:
Kotaniementie['Timestamp'] = Kotaniementie['Timestamp'].dt.round("min")
Kotaniementie.shape

(330119, 6)

In [43]:
Kotaniementie.shape

(330119, 6)

In [44]:
# Check for duplicates
Kotaniementie.duplicated().sum()

np.int64(84896)

In [45]:
# printing the duplicated rows
duplicates = Kotaniementie[Kotaniementie.duplicated()]
duplicates

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
3090,2021-01-22 17:29:00,0.7,0.012,267.12,0.0,70
3091,2021-01-22 17:29:00,0.7,0.012,267.12,0.0,70
3092,2021-01-22 17:29:00,0.7,0.012,267.12,0.0,70
3093,2021-01-22 17:29:00,0.7,0.012,267.12,0.0,70
3094,2021-01-22 17:29:00,0.7,0.012,267.12,0.0,70
...,...,...,...,...,...,...
325422,2025-08-16 19:59:00,0.0,0.000,147.05,0.0,0
325423,2025-08-16 19:59:00,0.0,0.000,147.05,0.0,0
325424,2025-08-16 19:59:00,0.0,0.000,147.05,0.0,0
325425,2025-08-16 19:59:00,0.0,0.000,147.05,0.0,0


In [46]:
# remove duplicates
Kotaniementie = Kotaniementie.drop_duplicates()
Kotaniementie.shape

(245223, 6)

In [47]:
Kotaniementie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245223 entries, 0 to 330118
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Timestamp                      245223 non-null  datetime64[ns]
 1   precipitationIntensity_mm_h    245223 non-null  float64       
 2   precipitationIntensity_mm_min  245223 non-null  float64       
 3   precipitationQuantityAbs_mm    245223 non-null  float64       
 4   precipitationQuantityDiff_mm   245223 non-null  float64       
 5   precipitationType              245223 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 13.1 MB


In [48]:
def check_timestamp_issues(df, Kotaniementie, expected_minutes=10):
    print(f"Checking: {Kotaniementie}")
    

    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')
    
    # Calculate time differences
    time_diffs = df['Timestamp'].diff()
    expected_gap = pd.Timedelta(minutes=expected_minutes)
    
    # Check for issues in Timestamps
    large_gaps = time_diffs[time_diffs > expected_gap * 1.1]
    duplicates = df['Timestamp'].duplicated().sum()
    time_travel = (time_diffs < pd.Timedelta(0)).sum()
    
    df['Date'] = df['Timestamp'].dt.date
    daily_counts = df['Date'].value_counts().sort_index()
   
    # Print results
    print(f"Number of gaps larger than {expected_minutes} minutes: {len(large_gaps)}")
    print(f"Large gaps: {large_gaps.max()}")
    print(f"Duplicates: {duplicates}")
    print(f"Time travel: {time_travel}")
    print(f"Daily counts: {daily_counts}")

Kotaniementie_info = check_timestamp_issues(Kotaniementie, "Kotaniementie WS100")

Checking: Kotaniementie WS100
Number of gaps larger than 10 minutes: 233
Large gaps: 3 days 08:31:00
Duplicates: 25
Time travel: 0
Daily counts: Date
2021-01-01    144
2021-01-02    144
2021-01-03    144
2021-01-04    144
2021-01-05    144
             ... 
2025-09-14    144
2025-09-15    144
2025-09-16    144
2025-09-17    144
2025-09-18     61
Name: count, Length: 1716, dtype: int64


In [49]:
# Let's find out start and end time of the data and the total days 
start_time = Kotaniementie["Timestamp"].min()
end_time = Kotaniementie["Timestamp"].max()
total_days = (end_time - start_time).days
print(f"Data starts from: {start_time}")
print(f"Data ends at: {end_time}")
print(f"Total days of data: {total_days} days")

Data starts from: 2021-01-01 00:04:00
Data ends at: 2025-09-18 10:06:00
Total days of data: 1721 days


In [50]:
Kotaniementie.describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,245223,245223.000000,245223.000000,245223.000000,245223.000000,245223.000000
mean,2023-05-12 21:11:22.071909888,0.058180,0.000976,160.491681,0.009545,8.102348
min,2021-01-01 00:04:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022-03-07 09:28:00,0.000000,0.000000,46.100000,0.000000,0.000000
50%,2023-05-14 10:50:00,0.000000,0.000000,120.430000,0.000000,0.000000
75%,2024-07-16 13:23:00,0.000000,0.000000,231.500000,0.000000,0.000000
max,2025-09-18 10:06:00,184.200000,3.070000,676.740000,13.680000,70.000000
std,NaN,0.852129,0.014202,149.323970,0.105035,21.696020


**Dry**

In [51]:
Kotaniementie[Kotaniementie['precipitationType'] == 0].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,215066,215066.000000,215066.000000,215066.000000,215066.000000,215066.0
mean,2023-05-17 19:42:08.262393856,0.000105,0.000002,160.655332,0.000280,0.0
min,2021-01-01 00:04:00,0.000000,0.000000,0.000000,0.000000,0.0
25%,2022-03-14 06:30:30,0.000000,0.000000,45.650000,0.000000,0.0
50%,2023-05-17 22:06:00,0.000000,0.000000,119.270000,0.000000,0.0
75%,2024-07-26 14:18:30,0.000000,0.000000,231.500000,0.000000,0.0
max,2025-09-18 10:06:00,0.100000,0.002000,676.740000,1.800000,0.0
std,NaN,0.003233,0.000065,150.943510,0.009626,0.0


**Snowy**

In [52]:
Kotaniementie[Kotaniementie['precipitationType'] == 70].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,12465,12465.000000,12465.000000,12465.000000,12465.000000,12465.0
mean,2023-03-13 20:34:06.344163840,0.227397,0.003843,151.748757,0.036881,70.0
min,2021-01-01 00:14:00,0.000000,0.000000,0.000000,0.000000,70.0
25%,2022-01-31 15:55:00,0.000000,0.000000,77.890000,0.010000,70.0
50%,2023-03-06 18:32:00,0.100000,0.002000,135.640000,0.020000,70.0
75%,2024-02-15 23:07:00,0.300000,0.005000,229.570000,0.060000,70.0
max,2025-05-14 11:40:00,4.200000,0.070000,665.070000,0.430000,70.0
std,NaN,0.270776,0.004485,93.404947,0.041083,0.0


**Rain**

In [53]:
Kotaniementie[Kotaniementie['precipitationType'] == 60].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,10970,10970.000000,10970.000000,10970.000000,10970.000000,10970.0
mean,2023-06-07 21:32:58.840473856,0.883036,0.014765,173.151450,0.140570,60.0
min,2021-01-03 06:44:00,0.000000,0.000000,0.000000,0.000000,60.0
25%,2022-05-12 21:17:30,0.000000,0.000000,34.092500,0.010000,60.0
50%,2023-07-08 10:01:00,0.100000,0.002000,94.950000,0.030000,60.0
75%,2024-08-22 15:32:30,0.500000,0.008000,246.065000,0.100000,60.0
max,2025-09-17 20:06:00,184.200000,3.070000,676.740000,13.680000,60.0
std,NaN,3.907744,0.065118,188.427415,0.470302,0.0


**Mixed**

In [54]:
Kotaniementie[(Kotaniementie['precipitationType'] >= 60) & (Kotaniementie['precipitationType'] <= 69)].describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,17692,17692.00000,17692.000000,17692.000000,17692.000000,17692.000000
mean,2023-04-25 03:31:53.698846720,0.64493,0.010792,164.662189,0.102919,62.985078
min,2021-01-01 05:04:00,0.00000,0.000000,0.000000,0.000000,60.000000
25%,2022-02-05 13:16:00,0.00000,0.000000,42.615000,0.010000,60.000000
50%,2023-05-15 02:00:30,0.10000,0.002000,115.295000,0.030000,60.000000
75%,2024-06-18 07:41:30,0.40000,0.007000,240.215000,0.070000,67.000000
max,2025-09-17 20:06:00,184.20000,3.070000,676.740000,13.680000,69.000000
std,NaN,3.09965,0.051651,160.871522,0.374592,3.861971


In [174]:
Kotaniementie.head()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2021-01-01 00:04:00,0.0,0.000,250.46,0.00,0
1,2021-01-01 00:14:00,0.1,0.002,250.46,0.00,70
2,2021-01-01 00:24:00,0.1,0.002,250.48,0.02,70
3,2021-01-01 00:34:00,0.1,0.002,250.49,0.01,70
4,2021-01-01 00:44:00,0.1,0.002,250.51,0.02,70


In [55]:
# Save cleaned data to new CSV
Kotaniementie.to_csv(r"D:\Fiveth\Marjetas_project\cleaned_datasets\wes100\df_Kotaniementie.csv", index=False)

<h3><center>Exploring Kotaniementie<h3>

In [56]:
# Kotaniementie is near the leakes, it might be effected by precipitation or humidity

In [57]:
df_Kotaniementie = Kotaniementie.copy() 
df_Kotaniementie['ptype'] = df_Kotaniementie['precipitationType'].apply(map_pt)
df_Kotaniementie['is_wet'] = df_Kotaniementie['ptype'].ne('dry')

In [58]:
summary = (df_Kotaniementie.groupby('ptype').agg(
          rows=('ptype', 'size'),
          # Converting records to hours, each record is 10 minutes 
          hours=('ptype', lambda s: s.size/6.0),
          # Total precipitation in mm for each type
          total_mm=('precipitationQuantityDiff_mm', 'sum'),
          # Average precipitation per record
          mean_mm_per_record=('precipitationQuantityDiff_mm', 'mean'),
          # Intensity distribution
          p25_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 25)),
          p50_intensity=('precipitationIntensity_mm_h', 'median'),
          p75_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 75)),
          p99_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 99)),).sort_values('total_mm', ascending=False).round(1))
summary

,rows,hours,total_mm,mean_mm_per_record,p25_intensity,p50_intensity,p75_intensity,p99_intensity
ptype,,,,,,,,
rain,10970,1828.3,1542.0,0.1,0.0,0.1,0.5,13.0
snow,12465,2077.5,459.7,0.0,0.0,0.1,0.3,1.1
mixed,6722,1120.3,278.8,0.0,0.1,0.2,0.3,1.4
dry,215066,35844.3,60.2,0.0,0.0,0.0,0.0,0.0


In [59]:
grouping = (df_Kotaniementie['is_wet'] != df_Kotaniementie['is_wet'].shift()).cumsum()

# Event aggregation:
events = (df_Kotaniementie[df_Kotaniementie['is_wet']]
    .assign(event_id=grouping[df_Kotaniementie['is_wet']])
    #Grouping  by each event
    .groupby('event_id').agg(
          # Event start time
        start=('Timestamp','min'),
        # Event end time
        end=('Timestamp','max'),
        duration_h=('Timestamp', lambda x: (x.max()-x.min()).total_seconds()/3600.0),
        total_mm=('precipitationQuantityDiff_mm','sum'),
        peak_intensity_mm_h=('precipitationIntensity_mm_h','max'),
        dominant_ptype=('ptype', lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0]),
        n=('ptype','size'))
    .sort_values(['total_mm','duration_h'], ascending=[False, False])
    .reset_index(drop=True))
event_stats = (events.groupby('dominant_ptype').agg(
              events=('dominant_ptype','size'),
              median_duration_h=('duration_h','median'),
              median_total_mm=('total_mm','median'),
              top_event_total_mm=('total_mm', lambda x: x.nlargest(10).sum()),
              total_mm=('total_mm','sum')).sort_values('total_mm', ascending=False).round(1))
events.head(25)

,start,end,duration_h,total_mm,peak_intensity_mm_h,dominant_ptype,n
0,2023-07-28 10:41:00,2023-07-29 15:52:00,29.183333,55.86,12.9,rain,176
1,2024-08-30 20:47:00,2024-08-30 23:17:00,2.500000,34.09,55.3,rain,16
2,2021-06-22 22:06:00,2021-06-22 23:36:00,1.500000,30.52,31.4,rain,10
3,2022-07-13 10:05:00,2022-07-13 11:05:00,1.000000,24.81,94.1,rain,7
4,2025-09-09 05:04:00,2025-09-09 06:24:00,1.333333,22.65,57.0,rain,9
5,2023-10-06 22:27:00,2023-10-08 03:07:00,28.666667,20.97,13.6,mixed,173
6,2023-08-28 16:02:00,2023-08-28 21:42:00,5.666667,20.75,22.2,rain,35
7,2023-06-30 20:55:00,2023-06-30 21:15:00,0.333333,20.58,184.2,rain,3
8,2025-06-24 16:20:00,2025-06-25 07:10:00,14.833333,19.55,15.8,rain,90
9,2024-09-13 09:30:00,2024-09-13 16:30:00,7.000000,19.33,11.8,rain,43


In [60]:
event_stats

,events,median_duration_h,median_total_mm,top_event_total_mm,total_mm
dominant_ptype,,,,,
rain,2414,0.2,0.0,263.9,1530.8
snow,2139,0.2,0.0,102.9,496.6
mixed,903,0.2,0.0,74.6,253.1


In [61]:
# Monthly aggregation
df_Kotaniementie['month'] = df_Kotaniementie['Timestamp'].dt.to_period('M')
monthly = (
    df_Kotaniementie.groupby(['month','ptype']).agg(total_mm=('precipitationQuantityDiff_mm','sum'), hours=('ptype', 'size')).reset_index())

# Hour of day 
df_Kotaniementie['hour'] = df_Kotaniementie['Timestamp'].dt.hour
hour_day = (df_Kotaniementie[df_Kotaniementie['is_wet']].groupby(['hour','ptype']).agg(mm_per_hour=('precipitationQuantityDiff_mm','sum'),
        records=('ptype','size')).reset_index())
monthly

,month,ptype,total_mm,hours
0,2021-01,dry,1.17,3112
1,2021-01,mixed,10.54,295
2,2021-01,rain,0.13,4
3,2021-01,snow,26.71,1028
4,2021-02,dry,0.53,3411
...,...,...,...,...
187,2025-07,rain,25.85,188
188,2025-08,dry,2.44,4233
189,2025-08,rain,53.98,233
190,2025-09,dry,0.40,2323


In [62]:
df_Kotaniementie['ptype_next'] = df_Kotaniementie['ptype'].shift(-1).fillna('dry')
transitions = (df_Kotaniementie.groupby(['ptype','ptype_next']).size().unstack(fill_value=0))

transitions_prob = transitions.div(transitions.sum(axis=1), axis=0).round(3)
transitions_prob


ptype_next,dry,mixed,rain,snow
ptype,,,,
dry,0.975,0.004,0.011,0.010
mixed,0.117,0.504,0.010,0.369
rain,0.219,0.007,0.759,0.015
snow,0.182,0.195,0.013,0.610


In [63]:
# Extremes Outliers per type
exterm = (df_Kotaniementie[df_Kotaniementie['is_wet']]
       .groupby('ptype')['precipitationIntensity_mm_h']
       .quantile([0.95,0.99,0.999]).unstack().round(1))


cut = df_Kotaniementie['precipitationIntensity_mm_h'].quantile(0.999)
spikes = df_Kotaniementie[df_Kotaniementie['precipitationIntensity_mm_h'] > cut]

# Optional: cap intensities to 99.9% to stabilize stats
df_Kotaniementie['intensity_capped'] = np.minimum(df_Kotaniementie['precipitationIntensity_mm_h'], cut)
df_Kotaniementie

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType,ptype,is_wet,month,hour,ptype_next,intensity_capped
0,2021-01-01 00:04:00,0.0,0.000,250.46,0.00,0,dry,False,2021-01,0,snow,0.0
1,2021-01-01 00:14:00,0.1,0.002,250.46,0.00,70,snow,True,2021-01,0,snow,0.1
2,2021-01-01 00:24:00,0.1,0.002,250.48,0.02,70,snow,True,2021-01,0,snow,0.1
3,2021-01-01 00:34:00,0.1,0.002,250.49,0.01,70,snow,True,2021-01,0,snow,0.1
4,2021-01-01 00:44:00,0.1,0.002,250.51,0.02,70,snow,True,2021-01,0,snow,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
330114,2025-09-18 09:26:00,0.0,0.000,208.16,0.00,0,dry,False,2025-09,9,dry,0.0
330115,2025-09-18 09:36:00,0.0,0.000,208.16,0.00,0,dry,False,2025-09,9,dry,0.0
330116,2025-09-18 09:46:00,0.0,0.000,208.16,0.00,0,dry,False,2025-09,9,dry,0.0
330117,2025-09-18 09:56:00,0.0,0.000,208.16,0.00,0,dry,False,2025-09,9,dry,0.0


In [64]:
report = (
    summary[['rows','hours','total_mm','mean_mm_per_record','p50_intensity','p99_intensity']]
    .rename(columns={
        'rows':'records',
        'hours':'hours_logged',
        'total_mm':'total_precip_mm',
        'mean_mm_per_record':'mean_mm_per_10min',
        'p50_intensity':'median_intensity_mm_h',
        'p99_intensity':'p99_intensity_mm_h'}).round(1))
report

,records,hours_logged,total_precip_mm,mean_mm_per_10min,median_intensity_mm_h,p99_intensity_mm_h
ptype,,,,,,
rain,10970,1828.3,1542.0,0.1,0.1,13.0
snow,12465,2077.5,459.7,0.0,0.1,1.1
mixed,6722,1120.3,278.8,0.0,0.2,1.4
dry,215066,35844.3,60.2,0.0,0.0,0.0


In [ ]:
data_Kotaniementie = Kotaniementie.copy()


def prepare(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()


    df["event"] = df['precipitationType'].apply(map_pt)

    # duration = time the row's event is "active" until the next timestamp
    dt = df["Timestamp"].shift(-1) - df["Timestamp"]
    dt_sec = dt.dt.total_seconds()

    # Fill last/invalid intervals with median positive step
    median_step = np.nanmedian(dt_sec[dt_sec > 0]) if np.any(dt_sec > 0) else 60.0
    dt_sec = np.where((dt_sec <= 0) | np.isnan(dt_sec), median_step, dt_sec)

    df["duration_h"] = dt_sec / 3600.0
    df.set_index("Timestamp", inplace=True)
    return df

data_Kotaniementie = prepare(data_Kotaniementie)

# Helper: aggregate by freq within range
FREQS = {"D": "Daily", "W-MON": "Weekly (Mon start)", "MS": "Monthly (Start)"}

def aggregate(df: pd.DataFrame, start: str, end: str, freq: str):
    sub = df.loc[start:end]
    g = pd.Grouper(freq=freq)

    # Duration by event
    dur_event = (
        sub.groupby([g, "event"])["duration_h"]
           .sum().reset_index()
    )

    # Precip by event
    mm_event = (
        sub.groupby([g, "event"])["precipitationQuantityDiff_mm"]
           .sum().reset_index()
    )

    # Total precip (for trend)
    mm_total = (
        sub["precipitationQuantityDiff_mm"]
           .resample(freq).sum().reset_index()
           .rename(columns={"precipitationQuantityDiff_mm":"total_mm", "Timestamp":"period"})
    )

    # Share % by event (time share)
    dur_total = dur_event.groupby("Timestamp")["duration_h"].sum().rename("dur_total")
    dur_event = dur_event.merge(dur_total, on="Timestamp", how="left")
    dur_event["share_%"] = np.where(dur_event["dur_total"] > 0,
                                    100 * dur_event["duration_h"] / dur_event["dur_total"], 0)

    # Round for display
    for col in ["duration_h", "share_%"]:
        dur_event[col] = dur_event[col].round(1)
    mm_event["precip_mm"] = mm_event["precipitationQuantityDiff_mm"].round(1)
    mm_event.drop(columns=["precipitationQuantityDiff_mm"], inplace=True)
    mm_total["total_mm"] = mm_total["total_mm"].round(1)

    return dur_event, mm_event, mm_total

# ------------------------------------------------------------
# 2) Dash app
app = Dash(__name__)
min_date = data_Kotaniementie.index.min().date()
max_date = data_Kotaniementie.index.max().date()

app.layout = html.Div(
    style={"fontFamily":"system-ui","padding":"16px","maxWidth":"1200px","margin":"0 auto"},
    children=[
        html.H2("WS100 – Dynamic Precipitation Dashboard"),
        html.Div([
            html.Div([
                html.Label("Date range"),
                dcc.DatePickerRange(
                    id="date-range", start_date=min_date, end_date=max_date,
                    min_date_allowed=min_date, max_date_allowed=max_date
                ),
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Aggregation"),
                dcc.Dropdown(
                    id="freq", value="MS",
                    options=[{"label":v, "value":k} for k,v in FREQS.items()],
                    clearable=False, style={"width":"220px"}
                )
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Metric"),
                dcc.RadioItems(
                    id="metric", value="duration",
                    options=[
                        {"label":"Duration (hours)", "value":"duration"},
                        {"label":"Precipitation (mm)", "value":"precip"},
                        {"label":"Time share (%)", "value":"share"},
                    ],
                    inline=True
                )
            ], style={"display":"inline-block"}),
        ], style={"marginBottom":"12px"}),

        dcc.Graph(id="stacked-graph", style={"height":"480px"}),
        dcc.Graph(id="total-graph", style={"height":"360px"}),

        html.H3("Summary (by period × event)"),
        dash_table.DataTable(
            id="summary-table",
            page_size=12,
            style_table={"overflowX":"auto"},
            style_cell={"padding":"6px", "fontSize":"14px"},
            style_header={"fontWeight":"600"}
        ),
        html.Div("Notes: Duration is computed from the time difference to the next timestamp; "
                 "‘Share’ is duration share per period. Precip negatives are clipped to 0.",
                 style={"marginTop":"8px","color":"#555"})
    ]
)


@app.callback(
    [Output("stacked-graph","figure"),
     Output("total-graph","figure"),
     Output("summary-table","data"),
     Output("summary-table","columns")],
    [Input("date-range","start_date"),
     Input("date-range","end_date"),
     Input("freq","value"),
     Input("metric","value")]
)
def update(start_date, end_date, freq, metric):
    dur_event, mm_event, mm_total = aggregate(data_Kotaniementie, start_date, end_date, freq)

    # Build stacked bar data & labels
    if metric == "duration":
        ycol, ytitle = "duration_h", "Duration (hours)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})
    elif metric == "precip":
        ycol, ytitle = "precip_mm", "Precipitation (mm)"
        df_plot = mm_event.rename(columns={"Timestamp":"period"})
    else:  # share
        ycol, ytitle = "share_%", "Time share (%)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})

    fig_stack = px.bar(
        df_plot, x="period", y=ycol, color="event",
        barmode="stack", text_auto=".1f",
        labels={"period":"Period", ycol:ytitle, "event":"Event"},
        title=f"{FREQS.get(freq, freq)} — {ytitle}"
    )
    if metric == "share":
        fig_stack.update_yaxes(range=[0, 100])

    # Total precip trend
    fig_total = px.line(
        mm_total, x="period", y="total_mm",
        markers=True, labels={"period":"Period","total_mm":"Total Precip (mm)"},
        title=f"{FREQS.get(freq, freq)} — Total Precipitation"
    )

    # Summary table: pivot by period × event with both duration and precip
    # Merge duration + precip into one wide table
    pivot_dur = dur_event.pivot_table(index="Timestamp", columns="event", values="duration_h", fill_value=0)
    pivot_mm  = mm_event.pivot_table(index="Timestamp", columns="event", values="precip_mm",  fill_value=0)
    out = pd.concat(
        {"Dur(h)": pivot_dur, "MM": pivot_mm},
        axis=1
    ).reset_index().rename(columns={"Timestamp":"period"})

    # Round nicely and format period
    out = out.sort_values("period")
    out["period"] = out["period"].dt.strftime("%Y-%m-%d")

    columns = [{"name": col if isinstance(col, str) else " / ".join(col), "id": col if isinstance(col, str) else " / ".join(col)}
               for col in out.columns]
    out.columns = [c["id"] for c in columns]
    data = out.round(1).to_dict("records")

    return fig_stack, fig_total, data, columns


# ------------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True)


In [66]:
# Since the Kotaniementie is closest to the Kakkovuorentie, but Kaakkovuorentie has less data, it is started to recording 2024-04-02 08:15:00
# So I am going to cut the Kotaniementie data from 2024-04-02 08:15:00 to match the Kaakkovuorentie data range
Kotaniementie["Timestamp"] = pd.to_datetime(Kotaniementie["Timestamp"],format="%Y-%m-%d %H:%M:%S")
Kotaniementie_cut = Kotaniementie[Kotaniementie["Timestamp"] >= "2024-04-02 08:15:00"]
Kotaniementie_cut = Kotaniementie_cut.sort_values(by="Timestamp").reset_index(drop=True)
Kotaniementie_cut.shape

(76447, 6)

In [67]:
# Let's find out start and end time of the data and the total days 
start_time = Kotaniementie_cut["Timestamp"].min()
end_time = Kotaniementie_cut["Timestamp"].max()
total_days = (end_time - start_time).days
print(f"Data starts from: {start_time}")
print(f"Data ends at: {end_time}")
print(f"Total days of data: {total_days} days")

Data starts from: 2024-04-02 08:20:00
Data ends at: 2025-09-18 10:06:00
Total days of data: 534 days


In [68]:
Kotaniementie_cut['ptype'] = Kotaniementie_cut['precipitationType'].apply(map_pt)
Kotaniementie_cut['is_wet'] = Kotaniementie_cut['ptype'].ne('dry')

In [69]:
summary = (Kotaniementie_cut.groupby('ptype').agg(
          rows=('ptype', 'size'),
          # Converting records to hours, each record is 10 minutes 
          hours=('ptype', lambda s: s.size/6.0),
          # Total precipitation in mm for each type
          total_mm=('precipitationQuantityDiff_mm', 'sum'),
          # Average precipitation per record
          mean_mm_per_record=('precipitationQuantityDiff_mm', 'mean'),
          # Intensity distribution
          p25_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 25)),
          p50_intensity=('precipitationIntensity_mm_h', 'median'),
          p75_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 75)),
          p99_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 99)),).sort_values('total_mm', ascending=False).round(1))
summary

,rows,hours,total_mm,mean_mm_per_record,p25_intensity,p50_intensity,p75_intensity,p99_intensity
ptype,,,,,,,,
rain,3785,630.8,525.4,0.1,0.0,0.1,0.4,13.4
snow,2465,410.8,96.0,0.0,0.0,0.1,0.4,1.2
mixed,1331,221.8,59.4,0.0,0.1,0.2,0.4,1.4
dry,68866,11477.7,21.0,0.0,0.0,0.0,0.0,0.0


In [70]:
grouping = (Kotaniementie_cut['is_wet'] != Kotaniementie_cut['is_wet'].shift()).cumsum()

# Event aggregation:
events = (Kotaniementie_cut[Kotaniementie_cut['is_wet']]
    .assign(event_id=grouping[Kotaniementie_cut['is_wet']])
    #Grouping  by each event
    .groupby('event_id').agg(
          # Event start time
        start=('Timestamp','min'),
        # Event end time
        end=('Timestamp','max'),
        duration_h=('Timestamp', lambda x: (x.max()-x.min()).total_seconds()/3600.0),
        total_mm=('precipitationQuantityDiff_mm','sum'),
        peak_intensity_mm_h=('precipitationIntensity_mm_h','max'),
        dominant_ptype=('ptype', lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0]),
        n=('ptype','size'))
    .sort_values(['total_mm','duration_h'], ascending=[False, False])
    .reset_index(drop=True))
event_stats = (events.groupby('dominant_ptype').agg(
              events=('dominant_ptype','size'),
              median_duration_h=('duration_h','median'),
              median_total_mm=('total_mm','median'),
              top_event_total_mm=('total_mm', lambda x: x.nlargest(10).sum()),
              total_mm=('total_mm','sum')).sort_values('total_mm', ascending=False).round(1))
events.head(25)
# Kotaniementie_cut

,start,end,duration_h,total_mm,peak_intensity_mm_h,dominant_ptype,n
0,2024-08-30 20:47:00,2024-08-30 23:17:00,2.500000,34.09,55.3,rain,16
1,2025-09-09 05:04:00,2025-09-09 06:24:00,1.333333,22.65,57.0,rain,9
2,2025-06-24 16:20:00,2025-06-25 07:10:00,14.833333,19.55,15.8,rain,90
3,2024-09-13 09:30:00,2024-09-13 16:30:00,7.000000,19.33,11.8,rain,43
4,2024-10-09 14:53:00,2024-10-10 03:33:00,12.666667,15.54,6.0,rain,77
5,2024-07-01 03:05:00,2024-07-01 08:45:00,5.666667,14.70,11.8,rain,35
6,2024-09-15 01:31:00,2024-09-15 07:11:00,5.666667,13.01,22.4,rain,35
7,2025-09-17 09:06:00,2025-09-17 12:56:00,3.833333,12.90,22.6,rain,24
8,2024-07-09 16:27:00,2024-07-09 16:57:00,0.500000,10.61,9.9,rain,4
9,2024-06-18 15:19:00,2024-06-18 15:59:00,0.666667,10.30,18.0,rain,5


In [71]:
# Save cut data 
Kotaniementie_cut.to_csv(r"D:\Fiveth\Marjetas_project\cleaned_datasets\wes100\df_Kotaniementie_cut.csv", index=False)

<h2><center>Saaritie<h2>

In [72]:
base = r"D:\Fiveth\Project_semester_automn_2025\Marjetas_Data\Marjetas_Data\JKL WS100\Data\Saaritie"
files = os.listdir(base)
files

['Saaritie_202101-202106.csv',
 'Saaritie_202107-202112.csv',
 'Saaritie_202201-202206.csv',
 'Saaritie_202207-202212.csv',
 'Saaritie_202301-202306.csv',
 'Saaritie_202307-202312.csv',
 'Saaritie_202401-202406.csv',
 'Saaritie_202407-202412.csv',
 'Saaritie_202501-202509.csv']

In [73]:
file_path = [os.path.join(base, f) for f in os.listdir(base) if f.endswith('.csv')]
Saaritie = pd.concat((pd.read_csv(f, sep=';') for f in file_path), ignore_index=True)


Saaritie["Timestamp"] = pd.to_datetime(Saaritie["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", Saaritie["Timestamp"].isna().sum())
# Sort the ds by timestamp
Saaritie = Saaritie.sort_values(by="Timestamp").reset_index(drop=True)
Saaritie.head()

NaT after parse: 0


,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2021-01-01 00:00:40,0.1,0.002,160.92,0.02,70
1,2021-01-01 00:10:46,0.3,0.005,160.94,0.02,70
2,2021-01-01 00:20:41,0.1,0.002,160.96,0.02,70
3,2021-01-01 00:30:50,0.0,0.000,160.97,0.01,69
4,2021-01-01 00:40:43,0.0,0.000,160.97,0.00,0


In [74]:
Saaritie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326985 entries, 0 to 326984
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Timestamp                      326985 non-null  datetime64[ns]
 1   precipitationIntensity_mm_h    326985 non-null  float64       
 2   precipitationIntensity_mm_min  326985 non-null  float64       
 3   precipitationQuantityAbs_mm    326985 non-null  float64       
 4   precipitationQuantityDiff_mm   326985 non-null  float64       
 5   precipitationType              326985 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 15.0 MB


In [75]:
Saaritie.shape

(326985, 6)

In [76]:
Saaritie.duplicated().sum()

np.int64(82397)

In [77]:
Saaritie['Timestamp'] = Saaritie['Timestamp'].dt.round("min")
Saaritie.shape

(326985, 6)

In [78]:
# printing the duplicated rows
duplicates = Saaritie[Saaritie.duplicated()]
duplicates

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
3742,2021-01-27 06:47:00,0.0,0.0,181.38,0.0,0
3743,2021-01-27 06:47:00,0.0,0.0,181.38,0.0,0
3744,2021-01-27 06:47:00,0.0,0.0,181.38,0.0,0
3745,2021-01-27 06:47:00,0.0,0.0,181.38,0.0,0
3746,2021-01-27 06:47:00,0.0,0.0,181.38,0.0,0
...,...,...,...,...,...,...
326739,2025-09-16 17:19:00,0.0,0.0,293.52,0.0,0
326740,2025-09-16 17:19:00,0.0,0.0,293.52,0.0,0
326741,2025-09-16 17:19:00,0.0,0.0,293.52,0.0,0
326742,2025-09-16 17:19:00,0.0,0.0,293.52,0.0,0


In [79]:
Saaritie = Saaritie.drop_duplicates()
Saaritie.shape

(239850, 6)

In [80]:
# Let's find out start and end time of the data and the total days 
start_time = Saaritie["Timestamp"].min()
end_time = Saaritie["Timestamp"].max()
total_days = (end_time - start_time).days
print(f"Data starts from: {start_time}")
print(f"Data ends at: {end_time}")
print(f"Total days of data: {total_days} days")

Data starts from: 2021-01-01 00:01:00
Data ends at: 2025-09-18 09:30:00
Total days of data: 1721 days


In [81]:
def check_timestamp_issues(df, Saaritie, expected_minutes=10):
    print(f"Checking: {Saaritie}")
    

    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')
    
    # Calculate time differences
    time_diffs = df['Timestamp'].diff()
    expected_gap = pd.Timedelta(minutes=expected_minutes)
    
    # Check for issues in Timestamps
    large_gaps = time_diffs[time_diffs > expected_gap * 1.1]
    duplicates = df['Timestamp'].duplicated().sum()
    time_travel = (time_diffs < pd.Timedelta(0)).sum()
    
    df['Date'] = df['Timestamp'].dt.date
    daily_counts = df['Date'].value_counts().sort_index()
   
    # Print results
    print(f"Number of gaps larger than {expected_minutes} minutes: {len(large_gaps)}")
    print(f"Large gaps: {large_gaps.max()}")
    print(f"Duplicates: {duplicates}")
    print(f"Time travel: {time_travel}")
    print(f"Daily counts: {daily_counts}")

Saaritie_info = check_timestamp_issues(Saaritie, "Saaritie WS100")

Checking: Saaritie WS100
Number of gaps larger than 10 minutes: 257
Large gaps: 36 days 01:32:00
Duplicates: 40
Time travel: 0
Daily counts: Date
2021-01-01    144
2021-01-02    144
2021-01-03    144
2021-01-04    144
2021-01-05    144
             ... 
2025-09-14    144
2025-09-15    144
2025-09-16    145
2025-09-17    144
2025-09-18     58
Name: count, Length: 1685, dtype: int64


In [82]:
# Save cleaned data to new CSV
Saaritie.to_csv(r"D:\Fiveth\Marjetas_project\cleaned_datasets\wes100\df_Saaritie.csv", index=False)

<h3><center>Exploring Saaritie<h3>

In [83]:
Saaritie.describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,239850,239850.000000,239850.000000,239850.000000,239850.000000,239850.000000
mean,2023-05-23 14:35:48.629393664,0.047481,0.000797,224.497101,0.008139,7.386442
min,2021-01-01 00:01:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022-04-01 18:12:30,0.000000,0.000000,74.650000,0.000000,0.000000
50%,2023-05-23 20:57:00,0.000000,0.000000,167.790000,0.000000,0.000000
75%,2024-07-15 09:07:30,0.000000,0.000000,319.530000,0.000000,0.000000
max,2025-09-18 09:30:00,58.900000,0.982000,738.100000,7.930000,70.000000
std,NaN,0.526625,0.008779,196.436974,0.080203,20.759304


In [84]:
df_Saaritie = Saaritie.copy() 

df_Saaritie['ptype'] = df_Saaritie['precipitationType'].apply(map_pt)
df_Saaritie['is_wet'] = df_Saaritie['ptype'].ne('dry')

In [85]:
summary = (df_Saaritie.groupby('ptype').agg(
          rows=('ptype', 'size'),
          # Converting records to hours, each record is 10 minutes 
          hours=('ptype', lambda s: s.size/6.0),
          # Total precipitation in mm for each type
          total_mm=('precipitationQuantityDiff_mm', 'sum'),
          # Average precipitation per record
          mean_mm_per_record=('precipitationQuantityDiff_mm', 'mean'),
          # Intensity distribution
          p25_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 25)),
          p50_intensity=('precipitationIntensity_mm_h', 'median'),
          p75_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 75)),
          p99_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 99)),).sort_values('total_mm', ascending=False).round(1))
summary

,rows,hours,total_mm,mean_mm_per_record,p25_intensity,p50_intensity,p75_intensity,p99_intensity
ptype,,,,,,,,
rain,10712,1785.3,1288.3,0.1,0.0,0.1,0.5,9.9
snow,8853,1475.5,311.0,0.0,0.0,0.1,0.3,1.1
mixed,7516,1252.7,297.2,0.0,0.1,0.2,0.3,1.5
dry,212769,35461.5,55.8,0.0,0.0,0.0,0.0,0.0


In [86]:
grouping = (df_Saaritie['is_wet'] != df_Saaritie['is_wet'].shift()).cumsum()

# Event aggregation:
events = (df_Saaritie[df_Saaritie['is_wet']]
    .assign(event_id=grouping[df_Saaritie['is_wet']])
    #Grouping  by each event
    .groupby('event_id').agg(
          # Event start time
        start=('Timestamp','min'),
        # Event end time
        end=('Timestamp','max'),
        duration_h=('Timestamp', lambda x: (x.max()-x.min()).total_seconds()/3600.0),
        total_mm=('precipitationQuantityDiff_mm','sum'),
        peak_intensity_mm_h=('precipitationIntensity_mm_h','max'),
        dominant_ptype=('ptype', lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0]),
        n=('ptype','size'))
    .sort_values(['total_mm','duration_h'], ascending=[False, False])
    .reset_index(drop=True))
event_stats = (events.groupby('dominant_ptype').agg(
              events=('dominant_ptype','size'),
              median_duration_h=('duration_h','median'),
              median_total_mm=('total_mm','median'),
              top_event_total_mm=('total_mm', lambda x: x.nlargest(10).sum()),
              total_mm=('total_mm','sum')).sort_values('total_mm', ascending=False).round(1))
events.head(25)


,start,end,duration_h,total_mm,peak_intensity_mm_h,dominant_ptype,n
0,2023-07-28 08:21:00,2023-07-29 15:51:00,31.500000,45.16,13.7,rain,190
1,2023-06-30 20:34:00,2023-06-30 22:34:00,2.000000,21.32,44.2,rain,13
2,2022-07-13 09:11:00,2022-07-13 10:51:00,1.666667,20.62,30.3,rain,11
3,2024-05-30 15:16:00,2024-05-30 16:56:00,1.666667,18.77,36.0,rain,11
4,2025-06-24 16:03:00,2025-06-25 07:23:00,15.333333,18.27,10.5,rain,93
5,2022-04-04 10:21:00,2022-04-06 03:31:00,41.166667,16.27,1.1,mixed,248
6,2021-06-22 20:38:00,2021-06-22 23:08:00,2.500000,16.13,43.8,rain,16
7,2024-06-13 06:22:00,2024-06-13 10:02:00,3.666667,15.41,19.5,rain,23
8,2024-10-09 14:35:00,2024-10-10 03:35:00,13.000000,15.38,13.1,rain,79
9,2023-10-06 22:05:00,2023-10-07 07:45:00,9.666667,15.26,5.3,rain,59


In [87]:
event_stats

,events,median_duration_h,median_total_mm,top_event_total_mm,total_mm
dominant_ptype,,,,,
rain,2304,0.2,0.0,200.2,1297.7
mixed,1212,0.2,0.0,63.6,311.0
snow,1484,0.2,0.0,71.7,287.8


In [88]:
df_Saaritie['ptype_next'] = df_Saaritie['ptype'].shift(-1).fillna('dry')
transitions = (df_Saaritie.groupby(['ptype','ptype_next']).size().unstack(fill_value=0))

transitions_prob = transitions.div(transitions.sum(axis=1), axis=0).round(3)
transitions_prob

ptype_next,dry,mixed,rain,snow
ptype,,,,
dry,0.977,0.005,0.011,0.007
mixed,0.144,0.583,0.008,0.266
rain,0.213,0.007,0.771,0.009
snow,0.185,0.218,0.011,0.586


In [89]:
report = (
    summary[['rows','hours','total_mm','mean_mm_per_record','p50_intensity','p99_intensity']]
    .rename(columns={
        'rows':'records',
        'hours':'hours_logged',
        'total_mm':'total_precip_mm',
        'mean_mm_per_record':'mean_mm_per_10min',
        'p50_intensity':'median_intensity_mm_h',
        'p99_intensity':'p99_intensity_mm_h'}).round(1))
report

,records,hours_logged,total_precip_mm,mean_mm_per_10min,median_intensity_mm_h,p99_intensity_mm_h
ptype,,,,,,
rain,10712,1785.3,1288.3,0.1,0.1,9.9
snow,8853,1475.5,311.0,0.0,0.1,1.1
mixed,7516,1252.7,297.2,0.0,0.2,1.5
dry,212769,35461.5,55.8,0.0,0.0,0.0


In [90]:
data_Saaritie = Saaritie.copy()


def prepare(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()


    df["event"] = df['precipitationType'].apply(map_pt)

    # duration = time the row's event is "active" until the next timestamp
    dt = df["Timestamp"].shift(-1) - df["Timestamp"]
    dt_sec = dt.dt.total_seconds()

    # Fill last/invalid intervals with median positive step
    median_step = np.nanmedian(dt_sec[dt_sec > 0]) if np.any(dt_sec > 0) else 60.0
    dt_sec = np.where((dt_sec <= 0) | np.isnan(dt_sec), median_step, dt_sec)

    df["duration_h"] = dt_sec / 3600.0
    df.set_index("Timestamp", inplace=True)
    return df

data_Saaritie = prepare(data_Saaritie)

# Helper: aggregate by freq within range
FREQS = {"D": "Daily", "W-MON": "Weekly (Mon start)", "MS": "Monthly (Start)"}

def aggregate(df: pd.DataFrame, start: str, end: str, freq: str):
    sub = df.loc[start:end]
    g = pd.Grouper(freq=freq)

    # Duration by event
    dur_event = (
        sub.groupby([g, "event"])["duration_h"]
           .sum().reset_index()
    )

    # Precip by event
    mm_event = (
        sub.groupby([g, "event"])["precipitationQuantityDiff_mm"]
           .sum().reset_index()
    )

    # Total precip (for trend)
    mm_total = (
        sub["precipitationQuantityDiff_mm"]
           .resample(freq).sum().reset_index()
           .rename(columns={"precipitationQuantityDiff_mm":"total_mm", "Timestamp":"period"})
    )

    # Share % by event (time share)
    dur_total = dur_event.groupby("Timestamp")["duration_h"].sum().rename("dur_total")
    dur_event = dur_event.merge(dur_total, on="Timestamp", how="left")
    dur_event["share_%"] = np.where(dur_event["dur_total"] > 0,
                                    100 * dur_event["duration_h"] / dur_event["dur_total"], 0)

    # Round for display
    for col in ["duration_h", "share_%"]:
        dur_event[col] = dur_event[col].round(1)
    mm_event["precip_mm"] = mm_event["precipitationQuantityDiff_mm"].round(1)
    mm_event.drop(columns=["precipitationQuantityDiff_mm"], inplace=True)
    mm_total["total_mm"] = mm_total["total_mm"].round(1)

    return dur_event, mm_event, mm_total

# ------------------------------------------------------------
# 2) Dash app
app = Dash(__name__)
min_date = data_Saaritie.index.min().date()
max_date = data_Saaritie.index.max().date()

app.layout = html.Div(
    style={"fontFamily":"system-ui","padding":"16px","maxWidth":"1200px","margin":"0 auto"},
    children=[
        html.H2("WS100 – Dynamic Precipitation Dashboard"),
        html.Div([
            html.Div([
                html.Label("Date range"),
                dcc.DatePickerRange(
                    id="date-range", start_date=min_date, end_date=max_date,
                    min_date_allowed=min_date, max_date_allowed=max_date
                ),
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Aggregation"),
                dcc.Dropdown(
                    id="freq", value="MS",
                    options=[{"label":v, "value":k} for k,v in FREQS.items()],
                    clearable=False, style={"width":"220px"}
                )
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Metric"),
                dcc.RadioItems(
                    id="metric", value="duration",
                    options=[
                        {"label":"Duration (hours)", "value":"duration"},
                        {"label":"Precipitation (mm)", "value":"precip"},
                        {"label":"Time share (%)", "value":"share"},
                    ],
                    inline=True
                )
            ], style={"display":"inline-block"}),
        ], style={"marginBottom":"12px"}),

        dcc.Graph(id="stacked-graph", style={"height":"480px"}),
        dcc.Graph(id="total-graph", style={"height":"360px"}),

        html.H3("Summary (by period × event)"),
        dash_table.DataTable(
            id="summary-table",
            page_size=12,
            style_table={"overflowX":"auto"},
            style_cell={"padding":"6px", "fontSize":"14px"},
            style_header={"fontWeight":"600"}
        ),
        html.Div("Notes: Duration is computed from the time difference to the next timestamp; "
                 "‘Share’ is duration share per period. Precip negatives are clipped to 0.",
                 style={"marginTop":"8px","color":"#555"})
    ]
)

# ------------------------------------------------------------
# 3) Callbacks
@app.callback(
    [Output("stacked-graph","figure"),
     Output("total-graph","figure"),
     Output("summary-table","data"),
     Output("summary-table","columns")],
    [Input("date-range","start_date"),
     Input("date-range","end_date"),
     Input("freq","value"),
     Input("metric","value")]
)
def update(start_date, end_date, freq, metric):
    dur_event, mm_event, mm_total = aggregate(data_Saaritie, start_date, end_date, freq)

    # Build stacked bar data & labels
    if metric == "duration":
        ycol, ytitle = "duration_h", "Duration (hours)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})
    elif metric == "precip":
        ycol, ytitle = "precip_mm", "Precipitation (mm)"
        df_plot = mm_event.rename(columns={"Timestamp":"period"})
    else:  # share
        ycol, ytitle = "share_%", "Time share (%)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})

    fig_stack = px.bar(
        df_plot, x="period", y=ycol, color="event",
        barmode="stack", text_auto=".1f",
        labels={"period":"Period", ycol:ytitle, "event":"Event"},
        title=f"{FREQS.get(freq, freq)} — {ytitle}"
    )
    if metric == "share":
        fig_stack.update_yaxes(range=[0, 100])

    # Total precip trend
    fig_total = px.line(
        mm_total, x="period", y="total_mm",
        markers=True, labels={"period":"Period","total_mm":"Total Precip (mm)"},
        title=f"{FREQS.get(freq, freq)} — Total Precipitation"
    )

    # Summary table: pivot by period × event with both duration and precip
    # Merge duration + precip into one wide table
    pivot_dur = dur_event.pivot_table(index="Timestamp", columns="event", values="duration_h", fill_value=0)
    pivot_mm  = mm_event.pivot_table(index="Timestamp", columns="event", values="precip_mm",  fill_value=0)
    out = pd.concat(
        {"Dur(h)": pivot_dur, "MM": pivot_mm},
        axis=1
    ).reset_index().rename(columns={"Timestamp":"period"})

    # Round nicely and format period
    out = out.sort_values("period")
    out["period"] = out["period"].dt.strftime("%Y-%m-%d")

    columns = [{"name": col if isinstance(col, str) else " / ".join(col), "id": col if isinstance(col, str) else " / ".join(col)}
               for col in out.columns]
    out.columns = [c["id"] for c in columns]
    data = out.round(1).to_dict("records")

    return fig_stack, fig_total, data, columns


# ------------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True)


<h2><center>Tahtiniementie<h2>

In [91]:
base = r"D:\Fiveth\Project_semester_automn_2025\Marjetas_Data\Marjetas_Data\JKL WS100\Data\Tähtiniementie"
files = os.listdir(base)
files

['Tähtiniementie_202101-202106.csv',
 'Tähtiniementie_202107-202112.csv',
 'Tähtiniementie_202201-202206.csv',
 'Tähtiniementie_202207-202212.csv',
 'Tähtiniementie_202301-202306.csv',
 'Tähtiniementie_202307-202312.csv',
 'Tähtiniementie_202401-202406.csv',
 'Tähtiniementie_202407-202412.csv',
 'Tähtiniementie_202501-202509.csv']

In [92]:
file_path = [os.path.join(base, f) for f in os.listdir(base) if f.endswith('.csv')]
Tahtiniementie = pd.concat((pd.read_csv(f, sep=';') for f in file_path), ignore_index=True)


Tahtiniementie["Timestamp"] = pd.to_datetime(Tahtiniementie["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", Tahtiniementie["Timestamp"].isna().sum())
# Sort the ds by timestamp
Tahtiniementie = Tahtiniementie.sort_values(by="Timestamp").reset_index(drop=True)
Tahtiniementie.head()

NaT after parse: 0


,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2021-01-01 00:00:43,0.1,0.002,33.73,0.02,70
1,2021-01-01 00:10:44,0.2,0.003,33.75,0.02,70
2,2021-01-01 00:20:43,0.0,0.000,33.76,0.01,70
3,2021-01-01 00:30:44,0.1,0.002,33.77,0.01,70
4,2021-01-01 00:40:44,0.2,0.003,33.78,0.01,70


In [93]:
Tahtiniementie.shape

(304151, 6)

In [94]:
Tahtiniementie['Timestamp'] = Tahtiniementie['Timestamp'].dt.round("min")
Tahtiniementie.shape

(304151, 6)

In [95]:
Tahtiniementie.duplicated().sum()

np.int64(61299)

In [96]:
# printing the duplicated rows
duplicates = Tahtiniementie[Tahtiniementie.duplicated()]
duplicates

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
4093,2021-01-29 16:28:00,0.0,0.0,67.69,0.0,0
4094,2021-01-29 16:28:00,0.0,0.0,67.69,0.0,0
4095,2021-01-29 16:28:00,0.0,0.0,67.69,0.0,0
4096,2021-01-29 16:28:00,0.0,0.0,67.69,0.0,0
4097,2021-01-29 16:28:00,0.0,0.0,67.69,0.0,0
...,...,...,...,...,...,...
296572,2025-07-27 18:08:00,0.0,0.0,275.94,0.0,0
296573,2025-07-27 18:08:00,0.0,0.0,275.94,0.0,0
296574,2025-07-27 18:08:00,0.0,0.0,275.94,0.0,0
296575,2025-07-27 18:08:00,0.0,0.0,275.94,0.0,0


In [97]:
Tahtiniementie = Tahtiniementie.drop_duplicates()
Tahtiniementie.shape

(242852, 6)

In [98]:
# Let's find out start and end time of the data and the total days 
start_time = Tahtiniementie["Timestamp"].min()
end_time = Tahtiniementie["Timestamp"].max()
total_days = (end_time - start_time).days
print(f"Data starts from: {start_time}")
print(f"Data ends at: {end_time}")
print(f"Total days of data: {total_days} days")

Data starts from: 2021-01-01 00:01:00
Data ends at: 2025-09-18 10:40:00
Total days of data: 1721 days


In [99]:
Tahtiniementie.describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,242852,242852.000000,242852.000000,242852.00000,242852.000000,242852.000000
mean,2023-05-11 15:06:28.520580352,0.057092,0.000957,69.50868,0.009410,7.802065
min,2021-01-01 00:01:00,0.000000,0.000000,0.00000,0.000000,0.000000
25%,2022-03-12 12:31:30,0.000000,0.000000,25.08000,0.000000,0.000000
50%,2023-05-10 13:31:00,0.000000,0.000000,56.99000,0.000000,0.000000
75%,2024-07-08 03:04:30,0.000000,0.000000,102.59000,0.000000,0.000000
max,2025-09-18 10:40:00,82.700000,1.378000,285.56000,8.620000,70.000000
std,NaN,0.674473,0.011242,57.22384,0.089645,21.297303


In [100]:
def check_timestamp_issues(df, Tahtiniementie, expected_minutes=10):
    print(f"Checking: {Tahtiniementie}")
    

    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')
    
    # Calculate time differences
    time_diffs = df['Timestamp'].diff()
    expected_gap = pd.Timedelta(minutes=expected_minutes)
    
    # Check for issues in Timestamps
    large_gaps = time_diffs[time_diffs > expected_gap * 1.1]
    duplicates = df['Timestamp'].duplicated().sum()
    time_travel = (time_diffs < pd.Timedelta(0)).sum()
    
    df['Date'] = df['Timestamp'].dt.date
    daily_counts = df['Date'].value_counts().sort_index()
   
    # Print results
    print(f"Number of gaps larger than {expected_minutes} minutes: {len(large_gaps)}")
    print(f"Large gaps: {large_gaps.max()}")
    print(f"Duplicates: {duplicates}")
    print(f"Time travel: {time_travel}")
    print(f"Daily counts: {daily_counts}")

Tahtiniementie_info = check_timestamp_issues(Tahtiniementie, "Tahtiniementie WS100")

Checking: Tahtiniementie WS100
Number of gaps larger than 10 minutes: 298
Large gaps: 7 days 03:37:00
Duplicates: 18
Time travel: 0
Daily counts: Date
2021-01-01    144
2021-01-02    144
2021-01-03    144
2021-01-04    144
2021-01-05    144
             ... 
2025-09-14    144
2025-09-15    143
2025-09-16    144
2025-09-17    144
2025-09-18     65
Name: count, Length: 1710, dtype: int64


In [101]:
# Save cleaned data to new CSV
Tahtiniementie.to_csv(r"D:\Fiveth\Marjetas_project\cleaned_datasets\wes100\df_Tahtiniementie.csv", index=False)

In [102]:
df_Tahtiniementie = Tahtiniementie.copy()
df_Tahtiniementie['ptype'] = df_Tahtiniementie['precipitationType'].apply(map_pt)
df_Tahtiniementie['is_wet'] = df_Tahtiniementie['ptype'].ne('dry')

In [103]:
summary = (df_Tahtiniementie.groupby('ptype').agg(
          rows=('ptype', 'size'),
          # Converting records to hours, each record is 10 minutes 
          hours=('ptype', lambda s: s.size/6.0),
          # Total precipitation in mm for each type
          total_mm=('precipitationQuantityDiff_mm', 'sum'),
          # Average precipitation per record
          mean_mm_per_record=('precipitationQuantityDiff_mm', 'mean'),
          # Intensity distribution
          p25_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 25)),
          p50_intensity=('precipitationIntensity_mm_h', 'median'),
          p75_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 75)),
          p99_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 99)),).sort_values('total_mm', ascending=False).round(1))
summary

,rows,hours,total_mm,mean_mm_per_record,p25_intensity,p50_intensity,p75_intensity,p99_intensity
ptype,,,,,,,,
rain,10921,1820.2,1442.5,0.1,0.0,0.1,0.5,10.7
snow,10375,1729.2,427.5,0.0,0.0,0.1,0.4,1.1
mixed,7575,1262.5,356.2,0.0,0.1,0.2,0.4,1.8
dry,213981,35663.5,59.0,0.0,0.0,0.0,0.0,0.0


In [104]:
grouping = (df_Tahtiniementie['is_wet'] != df_Tahtiniementie['is_wet'].shift()).cumsum()

# Event aggregation:
events = (df_Tahtiniementie[df_Tahtiniementie['is_wet']]
    .assign(event_id=grouping[df_Tahtiniementie['is_wet']])
    #Grouping  by each event
    .groupby('event_id').agg(
          # Event start time
        start=('Timestamp','min'),
        # Event end time
        end=('Timestamp','max'),
        duration_h=('Timestamp', lambda x: (x.max()-x.min()).total_seconds()/3600.0),
        total_mm=('precipitationQuantityDiff_mm','sum'),
        peak_intensity_mm_h=('precipitationIntensity_mm_h','max'),
        dominant_ptype=('ptype', lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0]),
        n=('ptype','size'))
    .sort_values(['total_mm','duration_h'], ascending=[False, False])
    .reset_index(drop=True))
event_stats = (events.groupby('dominant_ptype').agg(
              events=('dominant_ptype','size'),
              median_duration_h=('duration_h','median'),
              median_total_mm=('total_mm','median'),
              top_event_total_mm=('total_mm', lambda x: x.nlargest(10).sum()),
              total_mm=('total_mm','sum')).sort_values('total_mm', ascending=False).round(1))
events.head(25)

,start,end,duration_h,total_mm,peak_intensity_mm_h,dominant_ptype,n
0,2023-07-28 07:14:00,2023-07-29 14:44:00,31.500000,41.44,14.8,rain,190
1,2021-08-25 05:06:00,2021-08-25 17:16:00,12.166667,29.03,13.5,rain,74
2,2023-10-06 22:00:00,2023-10-07 13:30:00,15.500000,23.43,8.9,mixed,94
3,2023-08-28 12:22:00,2023-08-28 22:52:00,10.500000,22.95,13.9,rain,64
4,2024-09-14 19:33:00,2024-09-15 06:33:00,11.000000,20.45,18.3,rain,67
5,2022-08-24 07:28:00,2022-08-24 08:48:00,1.333333,19.75,56.8,rain,9
6,2021-06-23 17:21:00,2021-06-23 19:31:00,2.166667,19.68,82.7,rain,14
7,2024-10-09 14:19:00,2024-10-10 04:49:00,14.500000,19.35,7.8,rain,88
8,2025-06-24 15:16:00,2025-06-25 04:17:00,13.016667,18.57,12.9,rain,79
9,2021-06-22 21:03:00,2021-06-22 22:41:00,1.633333,18.53,55.3,rain,10


In [105]:
df = Tahtiniementie.copy()

def map_event(code: int):
    # Your rules
    if code == 0:
        return "None"
    if code == 60:
        return "Rain"
    if 61 <= code <= 69:
        return "Mix"
    if code == 70:
        return "Snow"
    return "Other"

def prepare(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp")
    df["precipitationQuantityDiff_mm"] = df["precipitationQuantityDiff_mm"].clip(lower=0)
    df["event"] = df["precipitationType"].fillna(0).astype(int).map(map_event)

    # duration = time the row's event is "active" until the next timestamp
    dt = df["Timestamp"].shift(-1) - df["Timestamp"]
    dt_sec = dt.dt.total_seconds()

    # Fill last/invalid intervals with median positive step
    median_step = np.nanmedian(dt_sec[dt_sec > 0]) if np.any(dt_sec > 0) else 60.0
    dt_sec = np.where((dt_sec <= 0) | np.isnan(dt_sec), median_step, dt_sec)

    df["duration_h"] = dt_sec / 3600.0
    df.set_index("Timestamp", inplace=True)
    return df

df = prepare(df)

# Helper: aggregate by freq within range
FREQS = {"D": "Daily", "W-MON": "Weekly (Mon start)", "MS": "Monthly (Start)"}

def aggregate(df: pd.DataFrame, start: str, end: str, freq: str):
    sub = df.loc[start:end]
    g = pd.Grouper(freq=freq)

    # Duration by event
    dur_event = (
        sub.groupby([g, "event"])["duration_h"]
           .sum().reset_index()
    )

    # Precip by event
    mm_event = (
        sub.groupby([g, "event"])["precipitationQuantityDiff_mm"]
           .sum().reset_index()
    )

    # Total precip (for trend)
    mm_total = (
        sub["precipitationQuantityDiff_mm"]
           .resample(freq).sum().reset_index()
           .rename(columns={"precipitationQuantityDiff_mm":"total_mm", "Timestamp":"period"})
    )

    # Share % by event (time share)
    dur_total = dur_event.groupby("Timestamp")["duration_h"].sum().rename("dur_total")
    dur_event = dur_event.merge(dur_total, on="Timestamp", how="left")
    dur_event["share_%"] = np.where(dur_event["dur_total"] > 0,
                                    100 * dur_event["duration_h"] / dur_event["dur_total"], 0)

    # Round for display
    for col in ["duration_h", "share_%"]:
        dur_event[col] = dur_event[col].round(1)
    mm_event["precip_mm"] = mm_event["precipitationQuantityDiff_mm"].round(1)
    mm_event.drop(columns=["precipitationQuantityDiff_mm"], inplace=True)
    mm_total["total_mm"] = mm_total["total_mm"].round(1)

    return dur_event, mm_event, mm_total

# ------------------------------------------------------------
# 2) Dash app
app = Dash(__name__)
min_date = df.index.min().date()
max_date = df.index.max().date()

app.layout = html.Div(
    style={"fontFamily":"system-ui","padding":"16px","maxWidth":"1200px","margin":"0 auto"},
    children=[
        html.H2("WS100 – Dynamic Precipitation Dashboard"),
        html.Div([
            html.Div([
                html.Label("Date range"),
                dcc.DatePickerRange(
                    id="date-range", start_date=min_date, end_date=max_date,
                    min_date_allowed=min_date, max_date_allowed=max_date
                ),
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Aggregation"),
                dcc.Dropdown(
                    id="freq", value="MS",
                    options=[{"label":v, "value":k} for k,v in FREQS.items()],
                    clearable=False, style={"width":"220px"}
                )
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Metric"),
                dcc.RadioItems(
                    id="metric", value="duration",
                    options=[
                        {"label":"Duration (hours)", "value":"duration"},
                        {"label":"Precipitation (mm)", "value":"precip"},
                        {"label":"Time share (%)", "value":"share"},
                    ],
                    inline=True
                )
            ], style={"display":"inline-block"}),
        ], style={"marginBottom":"12px"}),

        dcc.Graph(id="stacked-graph", style={"height":"480px"}),
        dcc.Graph(id="total-graph", style={"height":"360px"}),

        html.H3("Summary (by period × event)"),
        dash_table.DataTable(
            id="summary-table",
            page_size=12,
            style_table={"overflowX":"auto"},
            style_cell={"padding":"6px", "fontSize":"14px"},
            style_header={"fontWeight":"600"}
        ),
        html.Div("Notes: Duration is computed from the time difference to the next timestamp; "
                 "‘Share’ is duration share per period. Precip negatives are clipped to 0.",
                 style={"marginTop":"8px","color":"#555"})
    ]
)

# ------------------------------------------------------------
# 3) Callbacks
@app.callback(
    [Output("stacked-graph","figure"),
     Output("total-graph","figure"),
     Output("summary-table","data"),
     Output("summary-table","columns")],
    [Input("date-range","start_date"),
     Input("date-range","end_date"),
     Input("freq","value"),
     Input("metric","value")]
)
def update(start_date, end_date, freq, metric):
    dur_event, mm_event, mm_total = aggregate(df, start_date, end_date, freq)

    # Build stacked bar data & labels
    if metric == "duration":
        ycol, ytitle = "duration_h", "Duration (hours)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})
    elif metric == "precip":
        ycol, ytitle = "precip_mm", "Precipitation (mm)"
        df_plot = mm_event.rename(columns={"Timestamp":"period"})
    else:  # share
        ycol, ytitle = "share_%", "Time share (%)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})

    fig_stack = px.bar(
        df_plot, x="period", y=ycol, color="event",
        barmode="stack", text_auto=".1f",
        labels={"period":"Period", ycol:ytitle, "event":"Event"},
        title=f"{FREQS.get(freq, freq)} — {ytitle}"
    )
    if metric == "share":
        fig_stack.update_yaxes(range=[0, 100])

    # Total precip trend
    fig_total = px.line(
        mm_total, x="period", y="total_mm",
        markers=True, labels={"period":"Period","total_mm":"Total Precip (mm)"},
        title=f"{FREQS.get(freq, freq)} — Total Precipitation"
    )

    # Summary table: pivot by period × event with both duration and precip
    # Merge duration + precip into one wide table
    pivot_dur = dur_event.pivot_table(index="Timestamp", columns="event", values="duration_h", fill_value=0)
    pivot_mm  = mm_event.pivot_table(index="Timestamp", columns="event", values="precip_mm",  fill_value=0)
    out = pd.concat(
        {"Dur(h)": pivot_dur, "MM": pivot_mm},
        axis=1
    ).reset_index().rename(columns={"Timestamp":"period"})

    # Round nicely and format period
    out = out.sort_values("period")
    out["period"] = out["period"].dt.strftime("%Y-%m-%d")

    columns = [{"name": col if isinstance(col, str) else " / ".join(col), "id": col if isinstance(col, str) else " / ".join(col)}
               for col in out.columns]
    out.columns = [c["id"] for c in columns]
    data = out.round(1).to_dict("records")

    return fig_stack, fig_total, data, columns

if __name__ == "__main__":
    app.run(debug=True)


<h2><center>Tuulimyllyntie<h2>

In [106]:
base = r"D:\Fiveth\Project_semester_automn_2025\Marjetas_Data\Marjetas_Data\JKL WS100\Data\Tuulimyllyntie"
files = os.listdir(base)
files

['Tuulimyllyntie_202101-202106.csv',
 'Tuulimyllyntie_202107-202112.csv',
 'Tuulimyllyntie_202201-202206.csv',
 'Tuulimyllyntie_202207-202212.csv',
 'Tuulimyllyntie_202301-202306.csv',
 'Tuulimyllyntie_202307-202312.csv',
 'Tuulimyllyntie_202401-202406.csv',
 'Tuulimyllyntie_202407-202412.csv',
 'Tuulimyllyntie_202501-202509.csv']

In [107]:
file_path = [os.path.join(base, f) for f in os.listdir(base) if f.endswith('.csv')]
Tuulimyllyntie = pd.concat((pd.read_csv(f, sep=';') for f in file_path), ignore_index=True)


Tuulimyllyntie["Timestamp"] = pd.to_datetime(Tuulimyllyntie["Timestamp"],format="%Y-%m-%d %H:%M:%S", errors="coerce")
print("NaT after parse:", Tuulimyllyntie["Timestamp"].isna().sum())
# Sort the ds by timestamp
Tuulimyllyntie = Tuulimyllyntie.sort_values(by="Timestamp").reset_index(drop=True)
Tuulimyllyntie.head()

NaT after parse: 0


,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2021-01-01 00:03:41,0.1,0.002,507.53,0.02,70
1,2021-01-01 00:13:42,0.0,0.000,507.54,0.01,0
2,2021-01-01 00:23:41,0.0,0.000,507.54,0.00,0
3,2021-01-01 00:33:41,0.0,0.000,507.54,0.00,0
4,2021-01-01 00:43:42,0.0,0.000,507.54,0.00,0


In [108]:
Tuulimyllyntie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340081 entries, 0 to 340080
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Timestamp                      340081 non-null  datetime64[ns]
 1   precipitationIntensity_mm_h    340081 non-null  float64       
 2   precipitationIntensity_mm_min  340081 non-null  float64       
 3   precipitationQuantityAbs_mm    340081 non-null  float64       
 4   precipitationQuantityDiff_mm   340081 non-null  float64       
 5   precipitationType              340081 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 15.6 MB


In [109]:
Tuulimyllyntie.shape

(340081, 6)

In [110]:
Tuulimyllyntie.duplicated().sum()

np.int64(90494)

In [111]:
Tuulimyllyntie['Timestamp'] = Tuulimyllyntie['Timestamp'].dt.round("min")
Tuulimyllyntie.shape

(340081, 6)

In [112]:
# printing the duplicated rows
duplicates = Tuulimyllyntie[Tuulimyllyntie.duplicated()]
duplicates

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
3081,2021-01-22 16:29:00,0.0,0.0,522.37,0.0,0
3082,2021-01-22 16:29:00,0.0,0.0,522.37,0.0,0
3083,2021-01-22 16:29:00,0.0,0.0,522.37,0.0,0
3084,2021-01-22 16:29:00,0.0,0.0,522.37,0.0,0
3085,2021-01-22 16:29:00,0.0,0.0,522.37,0.0,0
...,...,...,...,...,...,...
333154,2025-07-31 16:52:00,0.0,0.0,217.28,0.0,0
333155,2025-07-31 16:52:00,0.0,0.0,217.28,0.0,0
333156,2025-07-31 16:52:00,0.0,0.0,217.28,0.0,0
333157,2025-07-31 16:52:00,0.0,0.0,217.28,0.0,0


In [113]:
Tuulimyllyntie = Tuulimyllyntie.drop_duplicates()
Tuulimyllyntie.shape

(244566, 6)

In [114]:
# Let's find out start and end time of the data and the total days 
start_time = Tuulimyllyntie["Timestamp"].min()
end_time = Tuulimyllyntie["Timestamp"].max()
total_days = (end_time - start_time).days
print(f"Data starts from: {start_time}")
print(f"Data ends at: {end_time}")
print(f"Total days of data: {total_days} days")

Data starts from: 2021-01-01 00:04:00
Data ends at: 2025-09-18 10:24:00
Total days of data: 1721 days


**Snowy**

In [115]:
Tuulimyllyntie[Tuulimyllyntie['precipitationType'] == 70]

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
0,2021-01-01 00:04:00,0.1,0.002,507.53,0.02,70
6,2021-01-01 01:04:00,0.1,0.002,507.55,0.01,70
7,2021-01-01 01:14:00,0.2,0.003,507.59,0.04,70
8,2021-01-01 01:24:00,0.1,0.002,507.61,0.02,70
9,2021-01-01 01:34:00,0.2,0.003,507.63,0.02,70
...,...,...,...,...,...,...
315217,2025-05-06 04:45:00,0.0,0.000,104.63,0.01,70
315218,2025-05-06 04:55:00,0.0,0.000,104.63,0.00,70
315290,2025-05-06 16:55:00,0.1,0.002,104.65,0.00,70
315304,2025-05-06 19:15:00,0.1,0.002,104.65,0.00,70


In [116]:
Tuulimyllyntie.describe()

,Timestamp,precipitationIntensity_mm_h,precipitationIntensity_mm_min,precipitationQuantityAbs_mm,precipitationQuantityDiff_mm,precipitationType
count,244566,244566.000000,244566.000000,244566.000000,244566.000000,244566.000000
mean,2023-05-16 15:08:41.866490624,0.062264,0.001045,312.834375,0.010218,9.500159
min,2021-01-01 00:04:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2022-03-15 05:24:30,0.000000,0.000000,78.060000,0.000000,0.000000
50%,2023-05-17 12:54:00,0.000000,0.000000,209.090000,0.000000,0.000000
75%,2024-07-17 04:59:30,0.000000,0.000000,520.510000,0.000000,0.000000
max,2025-09-18 10:24:00,96.600000,1.610000,979.510000,9.630000,70.000000
std,NaN,0.766823,0.012780,283.752413,0.093514,23.029367


In [117]:
def check_timestamp_issues(df, Tuulimyllyntie, expected_minutes=10):
    print(f"Checking: {Tuulimyllyntie}")
    

    df = df.copy()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df = df.sort_values('Timestamp')
    
    # Calculate time differences
    time_diffs = df['Timestamp'].diff()
    expected_gap = pd.Timedelta(minutes=expected_minutes)
    
    # Check for issues in Timestamps
    large_gaps = time_diffs[time_diffs > expected_gap * 1.1]
    duplicates = df['Timestamp'].duplicated().sum()
    time_travel = (time_diffs < pd.Timedelta(0)).sum()
    
    df['Date'] = df['Timestamp'].dt.date
    daily_counts = df['Date'].value_counts().sort_index()
   
    # Print results
    print(f"Number of gaps larger than {expected_minutes} minutes: {len(large_gaps)}")
    print(f"Large gaps: {large_gaps.max()}")
    print(f"Duplicates: {duplicates}")
    print(f"Time travel: {time_travel}")
    print(f"Daily counts: {daily_counts}")

Tuulimyllyntie_info = check_timestamp_issues(Tuulimyllyntie, "Tuulimyllyntie WS100")

Checking: Tuulimyllyntie WS100
Number of gaps larger than 10 minutes: 218
Large gaps: 9 days 23:12:00
Duplicates: 24
Time travel: 0
Daily counts: Date
2021-01-01    144
2021-01-02    144
2021-01-03    144
2021-01-04    144
2021-01-05    144
             ... 
2025-09-14    144
2025-09-15    144
2025-09-16    144
2025-09-17    144
2025-09-18     63
Name: count, Length: 1712, dtype: int64


In [118]:
df = Tuulimyllyntie.copy().sort_values("Timestamp").set_index("Timestamp")

# 1) Check basic stats for the core rain columns
print("=== Basic stats ===")
print(df[["precipitationQuantityAbs_mm",
          "precipitationQuantityDiff_mm",
          "precipitationIntensity_mm_h"]].describe())

# 2) Check if Abs is mostly monotonic (allowing small resets)
abs_diff = df["precipitationQuantityAbs_mm"].diff()

print("\n=== Abs diff stats ===")
print(abs_diff.describe())
print("Negative diffs count:", (abs_diff < -0.01).sum())
print("Big positive diffs (>= 10 mm):", (abs_diff >= 10).sum())

# 3) Compare Abs-diff vs. provided Diff
compare = pd.DataFrame({
    "abs_diff": abs_diff,
    "diff_col": df["precipitationQuantityDiff_mm"]
}).dropna()

compare["delta"] = (compare["abs_diff"] - compare["diff_col"]).abs()

print("\n=== Diff consistency ===")
print("Median |abs_diff - diff_col|:", compare["delta"].median())
print("95% percentile:", compare["delta"].quantile(0.95))
print("Max:", compare["delta"].max())

# 4) Look at a few rows where they differ a lot
suspicious = compare[compare["delta"] > 0.1].head(20)
print("\n=== Example mismatches (if any) ===")
print(suspicious)


=== Basic stats ===
       precipitationQuantityAbs_mm  precipitationQuantityDiff_mm  \
count                244566.000000                 244566.000000   
mean                    312.834375                      0.010218   
std                     283.752413                      0.093514   
min                       0.000000                      0.000000   
25%                      78.060000                      0.000000   
50%                     209.090000                      0.000000   
75%                     520.510000                      0.000000   
max                     979.510000                      9.630000   

       precipitationIntensity_mm_h  
count                244566.000000  
mean                      0.062264  
std                       0.766823  
min                       0.000000  
25%                       0.000000  
50%                       0.000000  
75%                       0.000000  
max                      96.600000  

=== Abs diff stats ===
count    2

In [119]:
# Save cleaned data to new CSV
Tuulimyllyntie.to_csv(r"D:\Fiveth\Marjetas_project\cleaned_datasets\wes100\df_Tuulimyllyntie.csv", index=False)

In [120]:
df_Tuulimyllyntie = Tuulimyllyntie.copy()
df_Tuulimyllyntie['ptype'] = df_Tuulimyllyntie['precipitationType'].apply(map_pt)
df_Tuulimyllyntie['is_wet'] = df_Tuulimyllyntie['ptype'].ne('dry')

In [121]:
summary = (df_Tuulimyllyntie.groupby('ptype').agg(
          rows=('ptype', 'size'),
          # Converting records to hours, each record is 10 minutes 
          hours=('ptype', lambda s: s.size/6.0),
          # Total precipitation in mm for each type
          total_mm=('precipitationQuantityDiff_mm', 'sum'),
          # Average precipitation per record
          mean_mm_per_record=('precipitationQuantityDiff_mm', 'mean'),
          # Intensity distribution
          p25_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 25)),
          p50_intensity=('precipitationIntensity_mm_h', 'median'),
          p75_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 75)),
          p99_intensity=('precipitationIntensity_mm_h', lambda x: np.percentile(x, 99)),).sort_values('total_mm', ascending=False).round(1))
summary

,rows,hours,total_mm,mean_mm_per_record,p25_intensity,p50_intensity,p75_intensity,p99_intensity
ptype,,,,,,,,
rain,15600,2600.0,1606.2,0.1,0.0,0.1,0.3,9.3
snow,10773,1795.5,422.1,0.0,0.0,0.1,0.4,1.2
mixed,9366,1561.0,404.0,0.0,0.1,0.2,0.3,1.5
dry,208827,34804.5,66.8,0.0,0.0,0.0,0.0,0.0


In [122]:
grouping = (df_Tuulimyllyntie['is_wet'] != df_Tuulimyllyntie['is_wet'].shift()).cumsum()

# Event aggregation:
events = (df_Tuulimyllyntie[df_Tuulimyllyntie['is_wet']]
    .assign(event_id=grouping[df_Tuulimyllyntie['is_wet']])
    #Grouping  by each event
    .groupby('event_id').agg(
          # Event start time
        start=('Timestamp','min'),
        # Event end time
        end=('Timestamp','max'),
        duration_h=('Timestamp', lambda x: (x.max()-x.min()).total_seconds()/3600.0),
        total_mm=('precipitationQuantityDiff_mm','sum'),
        peak_intensity_mm_h=('precipitationIntensity_mm_h','max'),
        dominant_ptype=('ptype', lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0]),
        n=('ptype','size'))
    .sort_values(['total_mm','duration_h'], ascending=[False, False])
    .reset_index(drop=True))
event_stats = (events.groupby('dominant_ptype').agg(
              events=('dominant_ptype','size'),
              median_duration_h=('duration_h','median'),
              median_total_mm=('total_mm','median'),
              top_event_total_mm=('total_mm', lambda x: x.nlargest(10).sum()),
              total_mm=('total_mm','sum')).sort_values('total_mm', ascending=False).round(1))
events.head(25)

,start,end,duration_h,total_mm,peak_intensity_mm_h,dominant_ptype,n
0,2023-07-28 08:29:00,2023-07-29 20:29:00,36.000000,101.15,29.1,rain,217
1,2024-08-30 20:26:00,2024-08-30 23:16:00,2.833333,40.07,53.9,rain,18
2,2024-09-14 19:59:00,2024-09-15 06:40:00,10.683333,23.28,42.7,rain,65
3,2023-06-30 20:52:00,2023-06-30 22:42:00,1.833333,22.64,81.6,rain,12
4,2025-08-02 14:32:00,2025-08-02 16:12:00,1.666667,19.86,60.1,rain,11
5,2023-10-06 22:19:00,2023-10-08 03:29:00,29.166667,18.41,3.8,mixed,176
6,2021-05-19 21:33:00,2021-05-20 00:53:00,3.333333,18.37,20.0,rain,21
7,2023-08-28 13:09:00,2023-08-29 03:00:00,13.850000,18.08,10.5,rain,84
8,2025-06-24 16:20:00,2025-06-25 07:30:00,15.166667,17.89,17.5,rain,92
9,2023-07-26 17:59:00,2023-07-26 19:59:00,2.000000,17.80,27.9,rain,13


In [123]:
# https://yle.fi/a/74-20043047

In [124]:
event_stats

,events,median_duration_h,median_total_mm,top_event_total_mm,total_mm
dominant_ptype,,,,,
rain,3443,0.2,0.0,296.0,1611.3
snow,1816,0.2,0.0,82.8,413.5
mixed,1285,0.3,0.0,96.2,407.4


In [125]:
df_Tuulimyllyntie['ptype_next'] = df_Tuulimyllyntie['ptype'].shift(-1).fillna('dry')
transitions = (df_Tuulimyllyntie.groupby(['ptype','ptype_next']).size().unstack(fill_value=0))

transitions_prob = transitions.div(transitions.sum(axis=1), axis=0).round(3)
transitions_prob


ptype_next,dry,mixed,rain,snow
ptype,,,,
dry,0.969,0.006,0.016,0.009
mixed,0.120,0.562,0.008,0.311
rain,0.218,0.006,0.765,0.011
snow,0.188,0.264,0.014,0.534


In [126]:
report = (
    summary[['rows','hours','total_mm','mean_mm_per_record','p50_intensity','p99_intensity']]
    .rename(columns={
        'rows':'records',
        'hours':'hours_logged',
        'total_mm':'total_precip_mm',
        'mean_mm_per_record':'mean_mm_per_10min',
        'p50_intensity':'median_intensity_mm_h',
        'p99_intensity':'p99_intensity_mm_h'}).round(1))
report

,records,hours_logged,total_precip_mm,mean_mm_per_10min,median_intensity_mm_h,p99_intensity_mm_h
ptype,,,,,,
rain,15600,2600.0,1606.2,0.1,0.1,9.3
snow,10773,1795.5,422.1,0.0,0.1,1.2
mixed,9366,1561.0,404.0,0.0,0.2,1.5
dry,208827,34804.5,66.8,0.0,0.0,0.0


In [127]:
df_Tuulimyllyntie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 244566 entries, 0 to 340080
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Timestamp                      244566 non-null  datetime64[ns]
 1   precipitationIntensity_mm_h    244566 non-null  float64       
 2   precipitationIntensity_mm_min  244566 non-null  float64       
 3   precipitationQuantityAbs_mm    244566 non-null  float64       
 4   precipitationQuantityDiff_mm   244566 non-null  float64       
 5   precipitationType              244566 non-null  int64         
 6   ptype                          244566 non-null  object        
 7   is_wet                         244566 non-null  bool          
 8   ptype_next                     244566 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 17.0+ MB


In [ ]:
df = Tuulimyllyntie.copy()

def map_event(code: int):
    # Your rules
    if code == 0:
        return "None"
    if code == 60:
        return "Rain"
    if 61 <= code <= 69:
        return "Mix"
    if code == 70:
        return "Snow"
    return "Other"

def prepare(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp")
    df["precipitationQuantityDiff_mm"] = df["precipitationQuantityDiff_mm"].clip(lower=0)
    df["event"] = df["precipitationType"].fillna(0).astype(int).map(map_event)

    # duration = time the row's event is "active" until the next timestamp
    dt = df["Timestamp"].shift(-1) - df["Timestamp"]
    dt_sec = dt.dt.total_seconds()

    # Fill last/invalid intervals with median positive step
    median_step = np.nanmedian(dt_sec[dt_sec > 0]) if np.any(dt_sec > 0) else 60.0
    dt_sec = np.where((dt_sec <= 0) | np.isnan(dt_sec), median_step, dt_sec)

    df["duration_h"] = dt_sec / 3600.0
    df.set_index("Timestamp", inplace=True)
    return df

df = prepare(df)

# Helper: aggregate by freq within range
FREQS = {"D": "Daily", "W-MON": "Weekly (Mon start)", "MS": "Monthly (Start)"}

def aggregate(df: pd.DataFrame, start: str, end: str, freq: str):
    sub = df.loc[start:end]
    g = pd.Grouper(freq=freq)

    # Duration by event
    dur_event = (
        sub.groupby([g, "event"])["duration_h"]
           .sum().reset_index()
    )

    # Precip by event
    mm_event = (
        sub.groupby([g, "event"])["precipitationQuantityDiff_mm"]
           .sum().reset_index()
    )

    # Total precip (for trend)
    mm_total = (
        sub["precipitationQuantityDiff_mm"]
           .resample(freq).sum().reset_index()
           .rename(columns={"precipitationQuantityDiff_mm":"total_mm", "Timestamp":"period"})
    )

    # Share % by event (time share)
    dur_total = dur_event.groupby("Timestamp")["duration_h"].sum().rename("dur_total")
    dur_event = dur_event.merge(dur_total, on="Timestamp", how="left")
    dur_event["share_%"] = np.where(dur_event["dur_total"] > 0,
                                    100 * dur_event["duration_h"] / dur_event["dur_total"], 0)

    # Round for display
    for col in ["duration_h", "share_%"]:
        dur_event[col] = dur_event[col].round(1)
    mm_event["precip_mm"] = mm_event["precipitationQuantityDiff_mm"].round(1)
    mm_event.drop(columns=["precipitationQuantityDiff_mm"], inplace=True)
    mm_total["total_mm"] = mm_total["total_mm"].round(1)

    return dur_event, mm_event, mm_total

# ------------------------------------------------------------
# 2) Dash app
app = Dash(__name__)
min_date = df.index.min().date()
max_date = df.index.max().date()

app.layout = html.Div(
    style={"fontFamily":"system-ui","padding":"16px","maxWidth":"1200px","margin":"0 auto"},
    children=[
        html.H2("WS100 – Dynamic Precipitation Dashboard"),
        html.Div([
            html.Div([
                html.Label("Date range"),
                dcc.DatePickerRange(
                    id="date-range", start_date=min_date, end_date=max_date,
                    min_date_allowed=min_date, max_date_allowed=max_date
                ),
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Aggregation"),
                dcc.Dropdown(
                    id="freq", value="MS",
                    options=[{"label":v, "value":k} for k,v in FREQS.items()],
                    clearable=False, style={"width":"220px"}
                )
            ], style={"display":"inline-block","marginRight":"24px"}),

            html.Div([
                html.Label("Metric"),
                dcc.RadioItems(
                    id="metric", value="duration",
                    options=[
                        {"label":"Duration (hours)", "value":"duration"},
                        {"label":"Precipitation (mm)", "value":"precip"},
                        {"label":"Time share (%)", "value":"share"},
                    ],
                    inline=True
                )
            ], style={"display":"inline-block"}),
        ], style={"marginBottom":"12px"}),

        dcc.Graph(id="stacked-graph", style={"height":"480px"}),
        dcc.Graph(id="total-graph", style={"height":"360px"}),

        html.H3("Summary (by period × event)"),
        dash_table.DataTable(
            id="summary-table",
            page_size=12,
            style_table={"overflowX":"auto"},
            style_cell={"padding":"6px", "fontSize":"14px"},
            style_header={"fontWeight":"600"}
        ),
        html.Div("Notes: Duration is computed from the time difference to the next timestamp; "
                 "‘Share’ is duration share per period. Precip negatives are clipped to 0.",
                 style={"marginTop":"8px","color":"#555"})
    ]
)

# ------------------------------------------------------------
# 3) Callbacks
@app.callback(
    [Output("stacked-graph","figure"),
     Output("total-graph","figure"),
     Output("summary-table","data"),
     Output("summary-table","columns")],
    [Input("date-range","start_date"),
     Input("date-range","end_date"),
     Input("freq","value"),
     Input("metric","value")]
)
def update(start_date, end_date, freq, metric):
    dur_event, mm_event, mm_total = aggregate(df, start_date, end_date, freq)

    # Build stacked bar data & labels
    if metric == "duration":
        ycol, ytitle = "duration_h", "Duration (hours)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})
    elif metric == "precip":
        ycol, ytitle = "precip_mm", "Precipitation (mm)"
        df_plot = mm_event.rename(columns={"Timestamp":"period"})
    else:  # share
        ycol, ytitle = "share_%", "Time share (%)"
        df_plot = dur_event.rename(columns={"Timestamp":"period"})

    fig_stack = px.bar(
        df_plot, x="period", y=ycol, color="event",
        barmode="stack", text_auto=".1f",
        labels={"period":"Period", ycol:ytitle, "event":"Event"},
        title=f"{FREQS.get(freq, freq)} — {ytitle}"
    )
    if metric == "share":
        fig_stack.update_yaxes(range=[0, 100])

    # Total precip trend
    fig_total = px.line(
        mm_total, x="period", y="total_mm",
        markers=True, labels={"period":"Period","total_mm":"Total Precip (mm)"},
        title=f"{FREQS.get(freq, freq)} — Total Precipitation"
    )

    # Summary table: pivot by period × event with both duration and precip
    # Merge duration + precip into one wide table
    pivot_dur = dur_event.pivot_table(index="Timestamp", columns="event", values="duration_h", fill_value=0)
    pivot_mm  = mm_event.pivot_table(index="Timestamp", columns="event", values="precip_mm",  fill_value=0)
    out = pd.concat(
        {"Dur(h)": pivot_dur, "MM": pivot_mm},
        axis=1
    ).reset_index().rename(columns={"Timestamp":"period"})

    # Round nicely and format period
    out = out.sort_values("period")
    out["period"] = out["period"].dt.strftime("%Y-%m-%d")

    columns = [{"name": col if isinstance(col, str) else " / ".join(col), "id": col if isinstance(col, str) else " / ".join(col)}
               for col in out.columns]
    out.columns = [c["id"] for c in columns]
    data = out.round(1).to_dict("records")

    return fig_stack, fig_total, data, columns

if __name__ == "__main__":
    app.run(debug=True)


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[128], line 149, in update(
    start_date='2021-01-01',
    end_date='2025-09-18',
    freq='MS',
    metric='duration'
)
    138 @app.callback(
    139     [Output("stacked-graph","figure"),
    140      Output("total-graph","figure"),
   (...)
    147 )
    148 def update(start_date, end_date, freq, metric):
--> 149     dur_event, mm_event, mm_total = aggregate(df, start_date, end_date, freq)
        df =                      precipitationIntensity_mm_h  \
Timestamp                                          
2021-01-01 00:04:00                          0.1   
2021-01-01 00:14:00                          0.0   
2021-01-01 00:24:00                          0.0   
2021-01-01 00:34:00                          0.0   
2021-01-01 00:44:00                          0.0   
...                                          ...   
2025-09-18 09

In [ ]:

location_dfs = {
    "Saaritie":        df_Saaritie,
    "Tuulimyllyntie":  df_Tuulimyllyntie,
    "Tähtiniementie":  df_Tahtiniementie,
    "Kaakkovuorentie":  df_Kaakkovuorentie,
    "Kotaniementie":   df_Kotaniementie,
}


def summarize_location(df):
    d = df.copy()
    d["mm"] = d["precipitationQuantityDiff_mm"].clip(lower=0)


    # event id for consecutive wet periods
    wet = d["ptype"].ne("dry")
    start = wet & ~wet.shift(fill_value=False)
    eid = start.cumsum().where(wet)

    ev = d[wet].groupby(eid, dropna=True).agg(
        start=("Timestamp","min"),
        end=("Timestamp","max"),
        total_mm=("mm","sum"),
        peak_mm_h=("precipitationIntensity_mm_h","max"),)
    if ev.empty:
        max_event_total = max_peak = med_dur = 0.0
    else:
        ev["dur_h"] = (ev["end"] - ev["start"]).dt.total_seconds()/3600.0
        max_event_total = ev["total_mm"].max()
        max_peak = ev["peak_mm_h"].max()
        med_dur = ev["dur_h"].median()

    by_type = d.groupby("ptype")["mm"].sum()
    tot = by_type.sum()
    rain_share = (by_type.get("rain",0)/tot*100) if tot else 0.0
    snow_share = (by_type.get("snow",0)/tot*100) if tot else 0.0
    mixed_share = (by_type.get("mixed",0)/tot*100) if tot else 0.0

    return {
        "total_mm": tot,
        "p99_mm_h": d["precipitationIntensity_mm_h"].quantile(0.99),
        "max_event_total_mm": max_event_total,
        "max_peak_mm_h": max_peak,
        "median_event_dur_h": med_dur,
        "rain_%": rain_share,
        "snow_%": snow_share,
        "mixed_%": mixed_share,
    }

#  Build tables
summary = pd.DataFrame({name: summarize_location(df) for name, df in location_dfs.items()}).T.round(1)
print("\n=== Per-location summary ===")
print(summary.to_string())

# Differences vs network mean
num_cols = ["total_mm","p99_mm_h","max_event_total_mm","max_peak_mm_h","median_event_dur_h","rain_%","snow_%","mixed_%"]
diff_mean = (summary[num_cols] - summary[num_cols].mean()).round(1)
print("\n=== Difference vs network mean (positive = above average) ===")
print(diff_mean.sort_values("total_mm", ascending=False).to_string())

# Differences vs a chosen reference (change if needed)
ref = "Kotaniementie"
diff_ref = (summary[num_cols].sub(summary.loc[ref, num_cols])).round(1)
print(f"\n=== Difference vs {ref} (positive = higher than {ref}) ===")
print(diff_ref.sort_values("total_mm", ascending=False).to_string())



=== Per-location summary ===
                 total_mm  p99_mm_h  max_event_total_mm  max_peak_mm_h  median_event_dur_h  rain_%  snow_%  mixed_%
Saaritie           1952.2       0.9                45.2           58.9                 0.2    66.0    15.9     15.2
Tuulimyllyntie     2499.1       1.0               101.2           96.6                 0.2    64.3    16.9     16.2
Tähtiniementie     2285.2       1.0                41.4           82.7                 0.2    63.1    18.7     15.6
Kaakkovuorentie     828.6       1.0                40.4          111.3                 0.2    75.8     7.4     14.3
Kotaniementie      2340.7       0.9                55.9          184.2                 0.2    65.9    19.6     11.9

=== Difference vs network mean (positive = above average) ===
                 total_mm  p99_mm_h  max_event_total_mm  max_peak_mm_h  median_event_dur_h  rain_%  snow_%  mixed_%
Tuulimyllyntie      517.9       0.0                44.4          -10.1                 0.0    -